# Prepare Environment

In [ ]:
!pip install transformers[torch] datasets pydantic==1.10 langchain[llms] openai tiktoken hdbscan wandb

In [ ]:
import os
import wandb

# Set API keys as environment variables
os.environ["OPENAI_API_KEY"] = "..."
os.environ["OPENAI_ORG"] = "..."
os.environ["WANDB_KEY"] = "..."

# Login to Weights & Biases for experiment tracking
wandb.login(key=os.environ["WANDB_KEY"])

In [ ]:
seed = 42

# Prepare Datasets

In [ ]:
# Import HuggingFace datasets package
import datasets
import pandas as pd
from sklearn.model_selection import train_test_split

# Define container class for datasets
class Dataset:

    def __init__(self, id, short_name, X_train, X_test, y_train, y_test, pos_class, context_train, context_test, train_zero_shot=False):
        self.id = id
        self.short_name = short_name
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.pos_class = pos_class # Class that counts as positive (for measuring precision, recall, and F1 score)
        self.context_train = context_train
        self.context_test = context_test
        self.train_zero_shot = train_zero_shot


    def __str__(self):
        return "Dataset(\n" + f"\tid='{self.id}'\n" + f"\tshort_name='{self.short_name}'\n" + f"\tcontext_train='{self.context_train}'\n" + f"\tcontext_test='{self.context_test}'\n" + f"\tclasses={list(pd.concat([self.y_train, self.y_test]).unique())}\n" + f"\tpos_class='{self.pos_class}'\n" + f"\ttrain_zero_shot={self.train_zero_shot}\n" + f"\tX_train.shape={self.X_train.shape}\n" + f"\tX_test.shape={self.X_test.shape}\n" + f"\ty_train.shape={self.y_train.shape}\n" + f"\ty_test.shape={self.y_test.shape}\n" + ")"


    def __repr__(self):
        return self.__str__()

In [ ]:
# Initialize list of datasets
dataset_list = {}

# Set overall sample size
n_train = 100
n_test = 100

## Fake Scientific Papers

In [ ]:
# Load the original training dataset
train = datasets.load_dataset("tum-nlp/IDMGSP", "train+gpt3", split="train")

# Randomly shuffle the dataset
train_shuffled = train.shuffle(seed=seed)

# Separate the different sources/generators
train_real = train_shuffled.filter(lambda x: x["src"] == "real")
train_scigen = train_shuffled.filter(lambda x: x["src"] == "scigen")
train_galactica = train_shuffled.filter(lambda x: x["src"] == "galactica")
train_gpt2 = train_shuffled.filter(lambda x: x["src"] == "gpt2")
train_gpt3 = train_shuffled.filter(lambda x: x["src"] == "gpt3")
train_chatgpt = train_shuffled.filter(lambda x: x["src"] == "chatgpt")

# Create one training dataset with alternating real and fake examples and equal number of examples from each fake source
train_fake = datasets.interleave_datasets([train_scigen, train_galactica, train_gpt2, train_gpt3, train_chatgpt])
data_train = datasets.interleave_datasets([train_real, train_fake])



# Load the original test datasets
test = datasets.load_dataset("tum-nlp/IDMGSP", "classifier_input", split="test")
ood_gpt3 = datasets.load_dataset("tum-nlp/IDMGSP", "ood_gpt3", split="test")

# Randomly shuffle the datasets
test_shuffled = test.shuffle(seed=seed)
test_gpt3 = ood_gpt3.shuffle(seed=seed)

# Separate the different sources/generators
test_real = test_shuffled.filter(lambda x: x["src"] == "real")
test_scigen = test_shuffled.filter(lambda x: x["src"] == "scigen")
test_galactica = test_shuffled.filter(lambda x: x["src"] == "galactica")
test_gpt2 = test_shuffled.filter(lambda x: x["src"] == "gpt2")
test_chatgpt = test_shuffled.filter(lambda x: x["src"] == "chatgpt")

# Create one test dataset with alternating real and fake examples and equal number of examples from each fake source
test_fake = datasets.interleave_datasets([test_scigen, test_galactica, test_gpt2, test_gpt3, test_chatgpt])
data_test = datasets.interleave_datasets([test_real, test_fake])



# Package dataset in a Dataset container
d = Dataset(
    id="tum-nlp/IDMGSP",
    short_name="papers",
    context_train="Examples are coming from a dataset with scientific papers that are either 'HUMAN-WRITTEN' (i.e., written by a real person) or 'MACHINE-GENERATED' (i.e., generated by a machine learning model). The downstream machine learning task is learning a classifier (e.g., Random Forest) that learns to distinguish 'HUMAN-WRITTEN' and 'MACHINE-GENERATED' scientific papers.",
    context_test="Examples are coming from a dataset with scientific papers that are either 'HUMAN-WRITTEN' (i.e., written by a real person) or 'MACHINE-GENERATED' (i.e., generated by a machine learning model). The downstream machine learning task is learning a classifier (e.g., Random Forest) that learns to distinguish 'HUMAN-WRITTEN' and 'MACHINE-GENERATED' scientific papers.",
    pos_class="MACHINE-GENERATED",
    X_train=data_train.to_pandas().head(n_train)[["title", "abstract", "introduction", "conclusion"]].rename(columns={"title": "Title", "abstract": "Abstract", "introduction": "Introduction", "conclusion": "Conclusion"}),
    X_test=data_test.to_pandas().head(n_test)[["title", "abstract", "introduction", "conclusion"]].rename(columns={"title": "Title", "abstract": "Abstract", "introduction": "Introduction", "conclusion": "Conclusion"}),
    y_train=data_train.to_pandas().head(n_train)["label"].map({0: "HUMAN-WRITTEN", 1: "MACHINE-GENERATED"}),
    y_test=data_test.to_pandas().head(n_test)["label"].map({0: "HUMAN-WRITTEN", 1: "MACHINE-GENERATED"})
)

# Add to dataset list
dataset_list[d.short_name] = d
d

## Fake News

In [ ]:
# Load the training dataset
data_train = datasets.load_dataset("GonzaloA/fake_news", split="train")

# Randomly shuffle the datasets
train_shuffled = data_train.shuffle(seed=seed)

# Separate the different classes (fake news and real news)
train_fake = train_shuffled.filter(lambda x: x["label"] == 0)
train_real = train_shuffled.filter(lambda x: x["label"] == 1)

# Create one dataset with alternating fake and real news examples
data_train = datasets.interleave_datasets([train_fake, train_real], stopping_strategy="first_exhausted")

# Convert to a Pandas DataFrame, split data and label columns, and limit to the training set size
df_train = data_train.to_pandas()
df_train = df_train.head(n_train)
df_train = df_train.rename(columns={"title": "Title", "text": "Text"})
df_train["label"] = df_train.apply(lambda row: "FAKE NEWS" if row["label"] == 0 else "REAL NEWS", axis=1)
X_train = df_train[["Title", "Text"]]
y_train = df_train["label"]



# Load the test dataset
data_test = datasets.load_dataset("GonzaloA/fake_news", split="test")

# Randomly shuffle the datasets
test_shuffled = data_test.shuffle(seed=seed)

# Separate the different classes (fake news and real news)
test_fake = test_shuffled.filter(lambda x: x["label"] == 0)
test_real = test_shuffled.filter(lambda x: x["label"] == 1)

# Create one dataset with alternating fake and real news examples
data_test = datasets.interleave_datasets([test_fake, test_real], stopping_strategy="first_exhausted")

# Convert to a Pandas DataFrame, split data and label columns, and limit to the test set size
df_test = data_test.to_pandas()
df_test = df_test.head(n_test)
df_test = df_test.rename(columns={"title": "Title", "text": "Text"})
df_test["label"] = df_test.apply(lambda row: "FAKE NEWS" if row["label"] == 0 else "REAL NEWS", axis=1)
X_test = df_test[["Title", "Text"]]
y_test = df_test["label"]



# Package dataset in a Dataset container
d = Dataset(
    id="GonzaloA/fake_news",
    short_name="fake-news-2",
    context_train="Examples are coming from a dataset with news articles that are either 'FAKE NEWS' or 'REAL NEWS'. The downstream machine learning task is learning a classifier (e.g., Random Forest) that learns to distinguish 'FAKE NEWS' and 'REAL NEWS' articles.",
    context_test="Examples are coming from a dataset with news articles that are either 'FAKE NEWS' or 'REAL NEWS'. The downstream machine learning task is learning a classifier (e.g., Random Forest) that learns to distinguish 'FAKE NEWS' and 'REAL NEWS' articles.",
    pos_class="FAKE NEWS",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test
)

# Add to dataset list
dataset_list[d.short_name] = d
d

## Hatespeech

In [ ]:
# Load the dataset
data = datasets.load_dataset("hate_speech18", split="train")

# Remove samples that rely on the context of previous messages
data = data.filter(lambda x: x["num_contexts"] == 0)

# Randomly shuffle the dataset
data_shuffled = data.shuffle(seed=seed)

# Separate the different classes (hate speech and no hate speech)
data_hate = data_shuffled.filter(lambda x: x["label"] == 1)
data_no_hate = data_shuffled.filter(lambda x: x["label"] == 0)

# Create one dataset with alternating hate and no hate examples
data = datasets.interleave_datasets([data_no_hate, data_hate], stopping_strategy="first_exhausted")

# Convert to a pandas dataframe
df = data.to_pandas()

# Drop unnecessary columns
df = df[["text", "label"]]

# Map the class numbers to natural-language class labels
df["label"] = df.apply(lambda row: "HATE SPEECH" if row["label"] == 1 else "NO HATE SPEECH", axis=1)

# Split data and label
X = df["text"]
y = df["label"]

# Split train and test sets
X_train = X.iloc[0:n_train]
y_train = y.iloc[0:n_train]
X_test = X.iloc[n_train:(n_train+n_test)]
y_test = y.iloc[n_train:(n_train+n_test)]

# Package dataset in a Dataset container
d = Dataset(
    id="hate_speech18",
    short_name="hate-speech",
    context_train="Examples are posts sampled from a white supremacist forum and are either 'HATE SPEECH' (when the posts contain hate speech) or 'NO HATE SPEECH' (when the posts do not contain hate speech). The downstream machine learning task is learning a classifier (e.g., Random Forest) that learns to distinguish 'HATE SPEECH' and 'NO HATE SPEECH' forum posts.",
    context_test="Examples are posts sampled from a white supremacist forum and are either 'HATE SPEECH' (when the posts contain hate speech) or 'NO HATE SPEECH' (when the posts do not contain hate speech). The downstream machine learning task is learning a classifier (e.g., Random Forest) that learns to distinguish 'HATE SPEECH' and 'NO HATE SPEECH' forum posts.",
    pos_class="HATE SPEECH",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test
)

# Add to dataset list
dataset_list[d.short_name] = d
d

## Reviews Amazon

In [ ]:
# Download the training dataset
print("Downloading training dataset ...")
data_train = datasets.load_dataset("amazon_polarity", split="train")

# Randomly shuffle the dataset
print("Preparing training dataset ...")
train_shuffled = data_train.shuffle(seed=seed)

# Separate positive and negative reviews
train_positive = train_shuffled.select(range(n_train*100)).filter(lambda x: x["label"] == 1)
train_negative = train_shuffled.select(range(n_train*100)).filter(lambda x: x["label"] == 0)

# Create one training dataset with alternating examples from both classes
df_train = datasets.interleave_datasets([train_positive, train_negative], stopping_strategy="first_exhausted").select(range(n_train)).to_pandas()

# Format the dataset
X_train = df_train.rename(columns={"title": "Title", "content": "Content"})[["Title", "Content"]]
y_train = df_train["label"].map({0: "NEGATIVE", 1: "POSITIVE"})



# Download the test dataset
print("Downloading test dataset ...")
data_test = datasets.load_dataset("amazon_polarity", split="test")

# Randomly shuffle the dataset
print("Preparing test dataset ...")
test_shuffled = data_test.shuffle(seed=seed)

# Separate positive and negative reviews
test_positive = test_shuffled.select(range(n_test*100)).filter(lambda x: x["label"] == 1)
test_negative = test_shuffled.select(range(n_test*100)).filter(lambda x: x["label"] == 0)

# Create one test dataset with alternating examples from both classes
df_test = datasets.interleave_datasets([test_positive, test_negative], stopping_strategy="first_exhausted").select(range(n_test)).to_pandas()

# Format the dataset
X_test = df_test.rename(columns={"title": "Title", "content": "Content"})[["Title", "Content"]]
y_test = df_test["label"].map({0: "NEGATIVE", 1: "POSITIVE"})



# Package dataset in a Dataset container
print("Packaging the dataset ...")
d = Dataset(
    id="amazon_polarity",
    short_name="reviews-amazon",
    context_train="Examples are product reviews from Amazon that are either 'NEGATIVE' (rating with 1 or 2 stars) or 'POSITIVE' (rating with 4 or 5 stars). The downstream machine learning task is learning a classifier (e.g., Random Forest) that learns to distinguish 'NEGATIVE' and 'POSITIVE' reviews.",
    context_test="Examples are product reviews from Amazon that are either 'NEGATIVE' (rating with 1 or 2 stars) or 'POSITIVE' (rating with 4 or 5 stars). The downstream machine learning task is learning a classifier (e.g., Random Forest) that learns to distinguish 'NEGATIVE' and 'POSITIVE' reviews.",
    pos_class="POSITIVE",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test
)

# Add to dataset list
dataset_list[d.short_name] = d
d

## Reviews Yelp

In [ ]:
# Download the training dataset
print("Downloading training dataset ...")
data_train = datasets.load_dataset("yelp_polarity", split="train")

# Randomly shuffle the dataset
print("Preparing training dataset ...")
train_shuffled = data_train.shuffle(seed=seed)

# Separate positive and negative reviews
train_positive = train_shuffled.select(range(n_train*100)).filter(lambda x: x["label"] == 1)
train_negative = train_shuffled.select(range(n_train*100)).filter(lambda x: x["label"] == 0)

# Create one training dataset with alternating examples from both classes
df_train = datasets.interleave_datasets([train_positive, train_negative], stopping_strategy="first_exhausted").select(range(n_train)).to_pandas()

# Format the dataset
X_train = df_train.rename(columns={"text": "Text"})[["Text"]]
y_train = df_train["label"].map({0: "NEGATIVE", 1: "POSITIVE"})



# Download the test dataset
print("Downloading test dataset ...")
data_test = datasets.load_dataset("yelp_polarity", split="test")

# Randomly shuffle the dataset
print("Preparing test dataset ...")
test_shuffled = data_test.shuffle(seed=seed)

# Separate positive and negative reviews
test_positive = test_shuffled.select(range(n_test*100)).filter(lambda x: x["label"] == 1)
test_negative = test_shuffled.select(range(n_test*100)).filter(lambda x: x["label"] == 0)

# Create one test dataset with alternating examples from both classes
df_test = datasets.interleave_datasets([test_positive, test_negative], stopping_strategy="first_exhausted").select(range(n_test)).to_pandas()

# Format the dataset
X_test = df_test.rename(columns={"text": "Text"})[["Text"]]
y_test = df_test["label"].map({0: "NEGATIVE", 1: "POSITIVE"})



# Package dataset in a Dataset container
print("Packaging the dataset ...")
d = Dataset(
    id="yelp_polarity",
    short_name="reviews-yelp",
    context_train="Examples are reviews from Yelp that are either 'NEGATIVE' (rating with 1 or 2 stars) or 'POSITIVE' (rating with 4 or 5 stars). The downstream machine learning task is learning a classifier (e.g., Random Forest) that learns to distinguish 'NEGATIVE' and 'POSITIVE' reviews.",
    context_test="Examples are reviews from Yelp that are either 'NEGATIVE' (rating with 1 or 2 stars) or 'POSITIVE' (rating with 4 or 5 stars). The downstream machine learning task is learning a classifier (e.g., Random Forest) that learns to distinguish 'NEGATIVE' and 'POSITIVE' reviews.",
    pos_class="POSITIVE",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test
)

# Add to dataset list
dataset_list[d.short_name] = d
d

## Sentiment

In [ ]:
# Download and combine training datasets in different languages
train_english = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split="train").filter(lambda x: x["label"] in [0, 2])
train_arabic = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "arabic", split="train").filter(lambda x: x["label"] in [0, 2])
train_french = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "french", split="train").filter(lambda x: x["label"] in [0, 2])
train_german = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "german", split="train").filter(lambda x: x["label"] in [0, 2])
train_hindi = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "hindi", split="train").filter(lambda x: x["label"] in [0, 2])
train_italian = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "italian", split="train").filter(lambda x: x["label"] in [0, 2])
train_portuguese = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "portuguese", split="train").filter(lambda x: x["label"] in [0, 2])
train_spanish = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "spanish", split="train").filter(lambda x: x["label"] in [0, 2])
df_train = datasets.interleave_datasets([train_english, train_arabic, train_french, train_german, train_hindi, train_italian, train_portuguese, train_spanish], stopping_strategy="first_exhausted").select(range(n_train)).to_pandas()

# Randomly shuffle the dataset
df_train = df_train.sample(frac=1, random_state=seed).reset_index(drop=True)

# Format the dataset
X_train = df_train["text"]
y_train = df_train["label"].map({0: "NEGATIVE", 2: "POSITIVE"})



# Download and combine test datasets in different languages
test_english = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english", split="test").filter(lambda x: x["label"] in [0, 2])
test_arabic = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "arabic", split="test").filter(lambda x: x["label"] in [0, 2])
test_french = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "french", split="test").filter(lambda x: x["label"] in [0, 2])
test_german = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "german", split="test").filter(lambda x: x["label"] in [0, 2])
test_hindi = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "hindi", split="test").filter(lambda x: x["label"] in [0, 2])
test_italian = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "italian", split="test").filter(lambda x: x["label"] in [0, 2])
test_portuguese = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "portuguese", split="test").filter(lambda x: x["label"] in [0, 2])
test_spanish = datasets.load_dataset("cardiffnlp/tweet_sentiment_multilingual", "spanish", split="test").filter(lambda x: x["label"] in [0, 2])
df_test = datasets.interleave_datasets([test_english, test_arabic, test_french, test_german, test_hindi, test_italian, test_portuguese, test_spanish], stopping_strategy="first_exhausted").select(range(n_test)).to_pandas()

# Randomly shuffle the dataset
df_test = df_test.sample(frac=1, random_state=seed).reset_index(drop=True)

# Format the dataset
X_test = df_test["text"]
y_test = df_test["label"].map({0: "NEGATIVE", 2: "POSITIVE"})



# Package dataset in a Dataset container
d = Dataset(
    id="cardiffnlp/tweet_sentiment_multilingual",
    short_name="sentiment",
    context_train="Examples are tweets in multiple languages which express either a 'NEGATIVE' or 'POSITIVE' sentiment. The downstream machine learning task is learning a classifier (e.g., Random Forest) that learns to distinguish 'NEGATIVE' and 'POSITIVE' tweets, independent of the language they are written in.",
    context_test="Examples are tweets in multiple languages which express either a 'NEGATIVE' or 'POSITIVE' sentiment. The downstream machine learning task is learning a classifier (e.g., Random Forest) that learns to distinguish 'NEGATIVE' and 'POSITIVE' tweets, independent of the language they are written in.",
    pos_class="POSITIVE",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test
)

# Add to dataset list
dataset_list[d.short_name] = d
d

# Define Models

## FELIX

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser, StructuredOutputParser, OutputFixingParser, ResponseSchema
from langchain.callbacks import get_openai_callback
from langchain.embeddings import OpenAIEmbeddings

from pydantic import BaseModel, Field
from typing import List

from hdbscan import HDBSCAN

import random
import time
import json
import re
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

from tqdm.notebook import tqdm


class PromptingTQDM(tqdm):
    def __init__(self, *args, **kwargs):
        super().__init__(bar_format="{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_inv_fmt}{postfix}]", *args, **kwargs)
        self.reset_stats()

    def reset_stats(self):
        # Initial stats
        self.tokens_in = 0
        self.tokens_out = 0
        self.costs = 0
        self.n_features = 0
        self.n_iter = 0

    def log_stats(self, tokens_in, tokens_out, cost, n_features):
        # Update statistics
        self.tokens_in += tokens_in
        self.tokens_out += tokens_out
        self.costs += cost
        self.n_features += n_features
        self.n_iter += 1

        # Calculate averages
        avg_tokens_in = self.tokens_in / self.n_iter
        avg_tokens_out = self.tokens_out / self.n_iter
        avg_cost = self.costs / self.n_iter

        # Update statistics in postfix string
        self.set_postfix_str(f"in {avg_tokens_in:.0f} tokens/{self.unit}, out {avg_tokens_out:.0f} tokens/{self.unit}, {avg_cost:.5f} USD/{self.unit}, {self.costs:.2f} USD total, {self.n_features:.0f} features total", refresh=True)


class NumericalFeature(BaseModel):
    name: str = Field(description="concise name of the feature")
    zero: str = Field(description="meaning of feature value of 0")
    ten: str = Field(description="meaning of feature value of 10")
    description: str = Field(description="short description of the meaning of this feature")


class NumericalFeatureSet(BaseModel):
    features: List[NumericalFeature] = Field(description="list of numeric features")


class CategoricalFeature(BaseModel):
    name: str = Field(description="concise name of the feature")
    possible_values: List[str] = Field(description="list of 2-5 different possible values allowed for this feature; the feature can take exactly one of these values at once")
    description: str = Field(description="short description of the meaning of this feature")


class CategoricalFeatureSet(BaseModel):
    features: List[CategoricalFeature] = Field(description="list of categorical features")


# A generic callback class that can be defined for FELIX and is called for every LLM request. Allows tracking of prompts, costs, and token consumption
class FELIXCallback:
    def __init__(self):
        pass

    def features_generated_for_pair(self, llm, example_a: str, example_b: str, label_a: str, label_b: str, system_message: str, prompt_message: str, llm_output: str, features: NumericalFeatureSet | CategoricalFeatureSet, total_cost: float, total_tokens: int, prompt_tokens: int, completion_tokens: int):
        pass

    def example_transformed(self, llm, example: str, feature_set: NumericalFeatureSet | CategoricalFeatureSet, system_message: str, prompt_message: str, llm_output: str, scores, total_cost: float, total_tokens: int, prompt_tokens: int, completion_tokens: int):
        pass

    def error_encountered(self, llm, system_message, prompt_message, llm_output, error):
        pass


class FELIX(BaseEstimator, TransformerMixin):

    def __init__(self, context=None, llm_name="gpt-3.5-turbo", llm_fallback="gpt-3.5-turbo-16k", llm_generation=None, llm_generation_fallback=None, llm_scoring=None, llm_scoring_fallback=None, llm_embeddings=None, temperature_generation=0.7, temperature_scoring=0.0, discrete_features=True, zero_shot=False, reschuffle_features=False, keep_outlier_features=False, openai_api_key=None, openai_organization=None, callback=None, verbose=False):
        self.context = context
        self.llm_name = llm_name
        self.llm_fallback = llm_fallback
        self.llm_generation = llm_generation if llm_generation else llm_name
        self.llm_generation_fallback = llm_generation_fallback if llm_generation_fallback else llm_fallback
        self.llm_scoring = llm_scoring if llm_scoring else llm_name
        self.llm_scoring_fallback = llm_scoring_fallback if llm_scoring_fallback else llm_fallback
        self.llm_embeddings = llm_embeddings
        self.temperature_generation = temperature_generation
        self.temperature_scoring = temperature_scoring
        self.discrete_features = discrete_features
        self.zero_shot = zero_shot
        self.reschuffle_features = reschuffle_features
        self.keep_noise = keep_outlier_features
        self.openai_api_key = openai_api_key
        self.openai_organization = openai_organization
        self.callback = callback
        self.verbose=verbose

        self._validate_class_variables(llm_name=True, context=True, temperature_generation=True, temperature_scoring=True, openai_api_key=True, openai_organization=True, verbose=True)


    def save_instance(self, filename):
        # Warn the user that certain parameters will not be serialized
        if self.callback:
            print("Warning: Parameter 'callback' is set but cannot be saved to instance.")
        if self._hdbscan:
            print("Warning: Parameters stored in '_hdbscan' cannot be saved to instance.")
        if self.openai_api_key or self.openai_organization:
            print("Warning: Parameters 'openai_api_key' and 'openai_organization' will not be saved to instance due to security reasons.")

        # Collect all parameters in a dictionary
        data = {
            "context": self.context,
            "llm_name": self.llm_name,
            "llm_fallback": self.llm_fallback,
            "llm_generation": self.llm_generation,
            "llm_generation_fallback": self.llm_generation_fallback,
            "llm_scoring": self.llm_scoring,
            "llm_scoring_fallback": self.llm_scoring_fallback,
            "llm_embeddings": self.llm_embeddings,
            "temperature_generation": self.temperature_generation,
            "temperature_scoring": self.temperature_scoring,
            "discrete_features": self.discrete_features,
            "zero_shot": self.zero_shot,
            "reschuffle_features": self.reschuffle_features,
            "keep_noise": self.keep_noise,
            "verbose": self.verbose,
            "full_feature_set": self._full_feature_set.json(),
            "features": self._features.json(),
            "feature_embeddings": self._feature_embeddings.tolist(),
            "cluster_labels": self._cluster_labels.tolist(),
            "duplicate_counter": self._global_duplicate_counter
        }

        # Convert the dictionary to a JSON string
        json_string = json.dumps(data, indent=4)

        # Save the JSON to a file
        with open(filename, "w") as f:
            f.write(json_string)


    def load_instance(self, filename):
        # Load the JSON from the file
        with open(filename, "r") as f:
            data = json.load(f)

        # Set all parameters again
        self.context = data.get("context")
        self.llm_name = data.get("llm_name")
        self.llm_fallback = data.get("llm_fallback")
        self.llm_generation = data.get("llm_generation")
        self.llm_generation_fallback = data.get("llm_generation_fallback")
        self.llm_scoring = data.get("llm_scoring")
        self.llm_scoring_fallback = data.get("llm_scoring_fallback")
        self.llm_embeddings = data.get("llm_embeddings")
        self.temperature_generation = data.get("temperature_generation")
        self.temperature_scoring = data.get("temperature_scoring")
        self.discrete_features = data.get("discrete_features")
        self.zero_shot = data.get("zero_shot") if "zero_shot" in data else False
        self.reschuffle_features = data.get("reschuffle_features") if "reschuffle_features" in data else False
        self.keep_noise = data.get("keep_noise") if "keep_noise" in data else True # Set to True for reverse compatibility (previous versions of FELIX always kept noisy features)
        self.verbose = data.get("verbose")
        self._full_feature_set = CategoricalFeatureSet.parse_raw(data.get("full_feature_set", [])) if self.discrete_features else NumericalFeatureSet.parse_raw(data.get("full_feature_set", []))
        self._features = CategoricalFeatureSet.parse_raw(data.get("features", [])) if self.discrete_features else NumericalFeatureSet.parse_raw(data.get("features", []))
        self._feature_embeddings = np.array(data.get("feature_embeddings"))
        self._cluster_labels = data.get("cluster_labels")
        self._global_duplicate_counter = data.get("duplicate_counter")

        # Set parameters to None that cannot be reconstructed
        self.callback = None
        self._hdbscan = None
        self.openai_api_key = None
        self.openai_organization = None


    def fit(self, X, y=None):
        # Validate input parameters and all required class variables
        X, y = self._validate_data(X, y)
        self._validate_class_variables(llm_name=True, context=True, temperature_generation=True, openai_api_key=True, openai_organization=True)

        # Generate features from the examples in X with class labels y
        self._full_feature_set = self.generate_features(X, y)

        # Consolidate features by removing duplicates
        self._features = self.consolidate_features(self._full_feature_set)

        return self


    def transform(self, X, y=None):
        # Validate input parameters and all required class variables
        X, y = self._validate_data(X, y, check_y=False)
        self._validate_class_variables(llm_name=True, context=True, temperature_scoring=True, openai_api_key=True, openai_organization=True)

        # Initialize the LLM for feature scoring
        llm = ChatOpenAI(model=self.llm_scoring, temperature=self.temperature_scoring, openai_api_key=self.openai_api_key, openai_organization=self.openai_organization)
        llm_fallback = ChatOpenAI(model=self.llm_scoring_fallback, temperature=self.temperature_scoring, openai_api_key=self.openai_api_key, openai_organization=self.openai_organization)

        # Score each example in X along the learned features in self._features
        X_transformed = []
        with PromptingTQDM(total=len(X), desc="Scoring examples", unit="prompt", disable=not self.verbose) as progress_bar:
            for i, example in X.items():
                scores, cb = self._transform_example(llm, example, self._features, llm_fallback)
                X_transformed.append(scores)
                if self.reschuffle_features:
                    # Randomly reorder the feature set after each scored example, if requested
                    random.shuffle(self._features.features)
                if self.verbose:
                    progress_bar.log_stats(cb.prompt_tokens, cb.completion_tokens, cb.total_cost, len(scores))
                    progress_bar.update(1)

        # Convert the scores into a Pandas DataFrame
        X_transformed = pd.DataFrame(X_transformed)

        return X_transformed


    def generate_features(self, X, y):
        # Validate input parameters and all required class variables
        X, y = self._validate_data(X, y)
        self._validate_class_variables(llm_name=True, context=True, temperature_generation=True, openai_api_key=True, openai_organization=True)

        # Initialize the LLM for feature generation
        llm = ChatOpenAI(model=self.llm_generation, temperature=self.temperature_generation, openai_api_key=self.openai_api_key, openai_organization=self.openai_organization)

        # Create pairs of examples by iterating over all class combinations and drawing examples from each class
        self._pairs = self._generate_pairs(X, y)

        # Generate features from the examples in X
        self._pairwise_feature_sets = []
        with PromptingTQDM(total=len(self._pairs), desc="Generating features", unit="prompt", disable=not self.verbose) as progress_bar:
            for example_a, example_b, label_a, label_b in self._pairs:
                features, cb = self._generate_features_for_pair(llm, example_a, example_b, label_a, label_b)
                self._pairwise_feature_sets.append(features)
                if self.verbose:
                    progress_bar.log_stats(cb.prompt_tokens, cb.completion_tokens, cb.total_cost, len(features.features))
                    progress_bar.update(1)

        # Flatten the lists of features generated
        flat_list = []
        for f in self._pairwise_feature_sets:
            flat_list = flat_list + f.features
        if self.discrete_features:
            full_feature_set = CategoricalFeatureSet(features=flat_list)
        else:
            full_feature_set = NumericalFeatureSet(features=flat_list)

        return full_feature_set


    def consolidate_features(self, feature_set):
        # Create text embeddings for each feature
        self._feature_embeddings = self._create_feature_embeddings(feature_set)

        # Cluster the embedded features using HDBSCAN to determine clusters of unique features
        self._cluster_labels = self._cluster_features(self._feature_embeddings)

        # Handle features identified as noise (i.e., features that were generated only once without another similar feature)
        if self.keep_noise:
            # Assign all features identified as noise (i.e., not part of a cluster) to their own cluster
            next_label = np.max(self._cluster_labels) + 1
            for i in range(len(self._cluster_labels)):
                if self._cluster_labels[i] == -1:
                    self._cluster_labels[i] = next_label
                    next_label += 1
        else:
            # If all features have been identified as noise, put them all in one cluster
            if (np.array(self._cluster_labels) == -1).all():
                self._cluster_labels = [0 for _ in self._cluster_labels]
                if self.verbose:
                    print("All features identified as noise. Treating them as one cluster")

            # Prune the feature set to only those features that are not identified as noise
            feature_set = feature_set.copy(deep=True)
            feature_set.features = [f for f, l in zip(feature_set.features, self._cluster_labels) if l != -1]
            self._feature_embeddings = np.array([e for e, l in zip(self._feature_embeddings, self._cluster_labels) if l != -1])
            self._cluster_labels = np.array([l for l in self._cluster_labels if l != -1])

        # Create a consolidated feature set with the most representative feature from each cluster
        self._features = self._select_representative_features(feature_set, self._feature_embeddings, self._cluster_labels)

        # Add number suffixes to remaining duplicate feature names to make sure that each feature name is unique
        self._features = self._ensure_unique_feature_names(self._features)

        print(f"Consolidated to {len(self._features.features)} features ({'incl.' if self.keep_noise else 'excl.'} noise)")

        return self._features


    def get_features_as_dataframe(self):
        # Validate that features have been learned already and that the learned feature set has the correct type
        if not self._features:
            raise ValueError("No features have been learned yet. Call fit() to learn a set of features.")
        if not isinstance(self._features, CategoricalFeatureSet) and not isinstance(self._features, NumericalFeatureSet):
            raise ValueError(f"Unexpected type '{type(self._features)}' of learned features. Should be either 'CategoricalFeatureSet' or 'NumericalFeatureSet'.")

        # Convert feature set to a Pandas DataFrame
        features_json = json.loads(self._features.json())["features"]
        df_features = pd.DataFrame(features_json)

        # Rename the DataFrame columns in a more readable format
        df_features.columns = df_features.columns.str.replace('_', ' ').str.title()

        return df_features


    def _validate_class_variables(self, llm_name=False, context=False, temperature_generation=False, temperature_scoring=False, openai_api_key=False, openai_organization=False, verbose=False):
        if temperature_generation:
            if not isinstance(self.temperature_generation, float):
                raise ValueError("temperature_generation must be of type float.")
            if not 0.0 <= self.temperature_generation <= 1.0:
                raise ValueError("temperature_generation must lie between 0.0 and 1.0.")

        if temperature_scoring:
            if not isinstance(self.temperature_scoring, float):
                raise ValueError("temperature_scoring must be of type float.")
            if not 0.0 <= self.temperature_scoring <= 1.0:
                raise ValueError("temperature_scoring must lie between 0.0 and 1.0.")

        if verbose:
            if not isinstance(self.verbose, bool):
                raise ValueError(f"verbose must be of type bool but is of type {type(self.verbose)}.")

        return True


    def _validate_data(self, X, y, check_X=True, check_y=True):
        if check_X:
            # Check if X is array-like
            if not (isinstance(X, (list, np.ndarray, pd.Series, pd.DataFrame))):
                raise ValueError("X must be an array-like object.")

            # Check if X has 1-2 dimensions
            if isinstance(X, (list, np.ndarray)):
                if len(np.shape(X)) > 2:
                    raise ValueError("X must have 1 or 2 dimensions.")
            elif isinstance(X, (pd.Series, pd.DataFrame)):
                if len(X.shape) > 2:
                    raise ValueError("X must have 1 or 2 dimensions.")

            # Check if X is empty
            if len(X) == 0:
                raise ValueError("X cannot be empty.")

            # Make sure X is a Pandas DataFrame
            X = pd.DataFrame(X)

            # Serialize DataFrame X to a 1-dimensional Series of text data
            X = self._serialize_dataframe(X)

        if check_y:
            # Check if y is array-like
            if not (isinstance(y, (list, np.ndarray, pd.Series, pd.DataFrame))):
                raise ValueError("y must be an array-like object.")

            # Check if y has exactly 1 dimension
            if isinstance(y, (list, np.ndarray)):
                if len(np.shape(y)) != 1:
                    raise ValueError("y can only have 1 dimension.")
            elif isinstance(y, (pd.Series, pd.DataFrame)):
                if len(y.shape) != 1:
                    raise ValueError("y can only have 1 dimension.")

            # Check if y is empty
            if len(y) == 0:
                raise ValueError("y cannot be empty.")

            # Make sure y is a Pandas object
            y = pd.Series(y)

            # Check if y contains at least two unique class labels
            if y.nunique() < 2:
                raise ValueError(f"y must contain at least two unique values but only has {y.nunique()} unique value.")

        if check_X and check_y:
            # Check if X and y have the same length
            if len(X) != len(y):
                raise ValueError(f"X and y must have the same length but X has length {len(X)} while y has length {len(y)}.")

        return X, y


    def _serialize_dataframe(self, df):
        # Check if df is a Pandas DataFrame
        if not isinstance(df, pd.DataFrame) and not isinstance(df, pd.Series):
            raise ValueError("df must be a Pandas DataFrame or Series.")

        # If df is already a Pandas Series, return it
        if isinstance(df, pd.Series):
            return df

        # Serialize DataFrame df to a 1-dimensional Series of text data
        if len(df.columns) == 1:
            # If df is a single-column DataFrame, keep it as a string Series
            return df[df.columns[0]].astype(str)
        else:
            # If Process DataFrame with multiple columns into the required format
            def serialize_row(row):
                # Replace NaN values with a placeholder, and cast each value to string
                items = [(col, str(val) if not pd.isnull(val) else "N/A") for col, val in row.items()]
                return "\n\n".join([f"{col}: {val}" for col, val in items])
            return df.apply(serialize_row, axis=1)


    def _generate_pairs(self, X, y):
        # Determine the set of unique classes
        classes = y.unique()

        # Create pairs of examples by iterating over all class combinations and drawing examples from each class
        pairs = []
        for i in range(len(classes)):
            for j in range(i+1, len(classes)):
                class_a = classes[i]
                class_b = classes[j]
                examples_class_a = X[y == class_a]
                examples_class_b = X[y == class_b]

                # Create pairs by iterating over the drawn examples from both classes.
                for k in range(max(len(examples_class_a), len(examples_class_b))):
                    # If one class runs out of examples, start with the first example of that class again. Alternate the order of the examples (AB, BA, AB, ...) (hence, the k % 2)
                    pairs.append((
                        examples_class_a.iloc[k % len(examples_class_a)] if k % 2 == 0 else examples_class_b.iloc[k % len(examples_class_b)],
                        examples_class_b.iloc[k % len(examples_class_b)] if k % 2 == 0 else examples_class_a.iloc[k % len(examples_class_a)],
                        class_a if k % 2 == 0 else class_b,
                        class_b if k % 2 == 0 else class_a
                    ))

        return pairs


    def _generate_features_for_pair(self, llm, example_a, example_b, label_a, label_b):
        # Check if any input parameters are None
        if llm == None:
            raise ValueError("llm cannot be None.")
        if example_a == None:
            raise ValueError("example_a cannot be None.")
        if example_b == None:
            raise ValueError("example_b cannot be None.")
        if label_a == None:
            raise ValueError("label_a cannot be None.")
        if label_b == None:
            raise ValueError("label_b cannot be None.")

        # Check if llm is an OpenAIChat model
        if not isinstance(llm, ChatOpenAI):
            raise ValueError(f"llm must be of type langchain.chat_models.openai.ChatOpenAI but found type {type(llm)}.")

        # Check if example_a and example_b are strings
        if not isinstance(example_a, str):
            raise ValueError(f"example_a must be of type str but found type {type(example_a)}.")
        if not isinstance(example_b, str):
            raise ValueError(f"example_b must be of type str but found type {type(example_b)}.")

        # Check if label_a and label_b are different
        if label_a == label_b:
            raise ValueError(f"label_a and label_b must be different (meaning that example_a and example_b must be from different classes) but found label_a=label_b={label_a}")

        # Check if all required class variables are set correctly
        self._validate_class_variables(context=True)

        # Define the output schema and parser (OutputFixingParser will try to fix the LLM output in case the PydanticOutputParser fails)
        if self.discrete_features:
            output_parser = OutputFixingParser.from_llm(
                parser=PydanticOutputParser(pydantic_object=CategoricalFeatureSet),
                llm=llm
            )
        else:
            output_parser = OutputFixingParser.from_llm(
                parser=PydanticOutputParser(pydantic_object=NumericalFeatureSet),
                llm=llm
            )

        # Define the prompt template for the system message with the context and overall task instructions
        system_template = SystemMessagePromptTemplate(prompt=PromptTemplate(
            template="You are a data scientist. Your goal is to engineer a set of features suitable to distinguish examples from different classes.{context}",
            input_variables=[],
            partial_variables={
                "context": " The concrete context is the following: " + self.context if self.context and self.context != "" else ""
            }
        ))

        # Define the prompt template for with the instructions to generate features
        if self.zero_shot:
            generation_template = HumanMessagePromptTemplate(prompt=PromptTemplate(
                template="""\
##### Instructions #####\n\n\
Create a list of features that would best distinguish examples of class '{label_a}' from examples of class '{label_b}'. \
Features can be anything from very specific (e.g., occurrences of a specific word) to very abstract (e.g., describing the logical flow of text). However, features should generalize even to new and previously unseen examples that may come from a slightly different domain. \
{feature_instructions}\n\n\
{format_instructions}""",
                input_variables=["label_a", "label_b"],
                partial_variables={
                    "feature_instructions": """Each feature should be categorical with at least 2 but no more than 5 different possible values. \
Possible values should be words but not numbers. \
For any given example, each feature should unambiguously take exactly one value out of the list of possible values. \
Thus, avoid features that could take two or more values for any example and ensure that there is always exactly one value that fits an example (if necessary, include fallback values such as 'Other', 'Neutral', or 'Not applicable').""" if self.discrete_features else """\
Each feature should be numeric and measured on a scale of 0 to 10. Therefore, indicate for each feature what 0 and 10 mean, respectively (e.g., 0 = "few", 10 = "many").""",
                    "format_instructions": output_parser.get_format_instructions()
                }
            ))
        else:
            generation_template = HumanMessagePromptTemplate(prompt=PromptTemplate(
                template="""\
##### {class_intro} '{label_a}' #####\n\n\
{example_a}\n\n\n\
##### {class_intro} '{label_b}' #####\n\n\
{example_b}\n\n\n\
##### Instructions #####\n\n\
Based on the above examples and their classes, create a list of features that would best distinguish examples of class '{label_a}' from examples of class '{label_b}'. \
Features can be anything from very specific (e.g., occurrences of a specific word) to very abstract (e.g., describing the logical flow of text). However, features should generalize even to new and previously unseen examples that may come from a slightly different domain. \
{feature_instructions}\n\n\
{format_instructions}""",
                input_variables=["label_a", "label_b", "example_a", "example_b"],
                partial_variables={
                    "class_intro": "The following example is from class",
                    "feature_instructions": """Each feature should be categorical with at least 2 but no more than 5 different possible values. \
Possible values should be words but not numbers. \
For any given example, each feature should unambiguously take exactly one value out of the list of possible values. \
Thus, avoid features that could take two or more values for any example and ensure that there is always exactly one value that fits an example (if necessary, include fallback values such as 'Other', 'Neutral', or 'Not applicable').""" if self.discrete_features else """\
Each feature should be numeric and measured on a scale of 0 to 10. Therefore, indicate for each feature what 0 and 10 mean, respectively (e.g., 0 = "few", 10 = "many").""",
                    "format_instructions": output_parser.get_format_instructions()
                }
            ))

        # Assemble a list of chat messages
        messages = [
            system_template.format(),
            generation_template.format(label_a=label_a, label_b=label_b) if self.zero_shot else generation_template.format(label_a=label_a, label_b=label_b, example_a=example_a, example_b=example_b)
        ]

        # Execute the prompt and parse the output
        with get_openai_callback() as cb:
            output = llm(messages).content
            features = output_parser.parse(output)

            # Convert all feature names to lower-case and replace spaces with underscores
            for f in features.features:
                f.name = f.name.lower().replace(" ", "_")

            # Report this LLM request to the callback for tracking and evaluation
            if self.callback:
                self.callback.features_generated_for_pair(llm, example_a, example_b, label_a, label_b, messages[0].content, messages[1].content, output, features, cb.total_cost, cb.total_tokens, cb.prompt_tokens, cb.completion_tokens)

            return features, cb


    def _create_feature_embeddings(self, feature_set):
        # Check if feature_set is of type FeatureSet
        if self.discrete_features and not isinstance(feature_set, CategoricalFeatureSet):
            raise ValueError(f"feature_set must be of type CategoricalFeatureSet but found {type(feature_set)}.")
        elif not self.discrete_features and not isinstance(feature_set, NumericalFeatureSet):
            raise ValueError(f"feature_set must be of type NumericalFeatureSet but found {type(feature_set)}.")
        for f in feature_set.features:
            if self.discrete_features:
                if not isinstance(f, CategoricalFeature):
                    raise ValueError(f"All features in feature_set.features must be of type CategoricalFeature but found an element of type {type(f)}: {f}.")
            else:
                if not isinstance(f, NumericalFeature):
                    raise ValueError(f"All features in feature_set.features must be of type NumericalFeature but found an element of type {type(f)}: {f}.")

        # Check if feature_set contains at least one feature
        if len(feature_set.features) == 0:
            raise ValueError(f"feature_set must contain at least 1 feature but found {len(feature_set.features)} features.")

        # Check if all required class variables are set correctly
        self._validate_class_variables(openai_api_key=True, openai_organization=True)

        # Instantiate OpenAI embeddings
        embeddings = OpenAIEmbeddings(
            model=self.llm_embeddings,
            openai_api_key=self.openai_api_key,
            openai_organization=self.openai_organization
        ) if self.llm_embeddings else OpenAIEmbeddings(
            openai_api_key=self.openai_api_key,
            openai_organization=self.openai_organization
        )

        # Create text embeddings for each feature
        with tqdm(total=1, desc="Creating feature embeddings", disable=not self.verbose) as progress_bar:
            feature_embeddings = embeddings.embed_documents([f.json() for f in feature_set.features])
            feature_embeddings = np.array(feature_embeddings)
            if self.verbose:
                progress_bar.update(1)

        return feature_embeddings


    def _cluster_features(self, feature_embeddings):
        # Check if feature_embeddings has the right format
        if not isinstance(feature_embeddings, np.ndarray):
            raise ValueError(f"feature_embeddings must be of type np.ndarray but found {type(feature_embeddings)}.")

        # Cluster embedded features using HDBSCAN to determine clusters of unique features
        with tqdm(total=1, desc="Clustering features", disable=not self.verbose) as progress_bar:
            np.random.seed(0)
            self._hdbscan = HDBSCAN(min_cluster_size=2, allow_single_cluster=True, cluster_selection_method="leaf", cluster_selection_epsilon=0.0, alpha=1.0)
            cluster_labels = self._hdbscan.fit_predict(feature_embeddings)
            if self.verbose:
                progress_bar.update(1)

        return cluster_labels


    def _select_representative_features(self, feature_set, feature_embeddings, cluster_labels):
        # Find the unique cluster labels
        unique_labels = np.unique(cluster_labels)

        # Initialize a list to hold the representative points of each cluster
        representative_features = []

        # Loop over each unique cluster label
        for label in unique_labels:
            # Get the points in this cluster
            cluster_points = np.array([(point, cluster_label, feature) for point, cluster_label, feature in zip(feature_embeddings, cluster_labels, feature_set.features) if cluster_label == label], dtype=object)

            # Calculate the centroid of these points
            centroid = np.mean([p[0] for p in cluster_points], axis=0)

            # Calculate the distance of each point from the centroid
            distances = cdist([centroid], [p[0] for p in cluster_points])[0]

            # Get the index of the closest point
            closest_point_idx = np.argmin(distances)

            # Get the closest point and add it to our list
            representative_features.append(cluster_points[closest_point_idx][2])

        # Consolidate all representative features into a new feature set
        if self.discrete_features:
            feature_set_consolidated = CategoricalFeatureSet(features=representative_features)
        else:
            feature_set_consolidated = NumericalFeatureSet(features=representative_features)

        return feature_set_consolidated


    def _ensure_unique_feature_names(self, feature_set):
        # Convert all feature names to lower-case and replace spaces with underscores
        for f in feature_set.features:
            f.name = f.name.lower().replace(" ", "_")

        # Count the number of duplicate feature names in the feature set
        self._global_duplicate_counter = 0

        # Iterate over all feature names and look for duplicates. If a duplicate is found, attach a number suffix ("_2", "_3", ...)
        for i in range(len(feature_set.features)):
            local_duplicate_counter = 0
            for j in range(i+1, len(feature_set.features)):
                if feature_set.features[i].name == feature_set.features[j].name:
                    self._global_duplicate_counter += 1
                    local_duplicate_counter += 1
                    feature_set.features[j].name = feature_set.features[j].name + "_" + str(local_duplicate_counter + 1)

        return feature_set


    def _transform_example(self, llm, example, feature_set, llm_fallback=None):
        # Define the response schemas to get a score for each feature
        response_schemas = []
        for f in feature_set.features:
            response_schemas.append(ResponseSchema(
                name=f.name.lower().replace(" ", "_"),
                description=f"'{f.description}' (value can be any of {str(f.possible_values)})",
                type="string"
            ) if self.discrete_features else ResponseSchema(
                name=f.name.lower().replace(" ", "_"),
                description=f"'{f.name.lower()}' (value from 0 to 10, 0 means '{f.zero}', 10 means '{f.ten}')",
                type="int"
            ))

        # Define the output schema and parser (OutputFixingParser will try to fix the LLM output in case the StructuredOutputParser fails)
        output_parser = OutputFixingParser.from_llm(
            parser=StructuredOutputParser.from_response_schemas(response_schemas),
            llm=llm_fallback if llm_fallback else llm # Note: Use fallback LLM (with larger context window) to be sure that output fixing does not fail because of a too small context window
        )

        # Define the prompt template for the system message with the context and overall task instructions
        system_template = SystemMessagePromptTemplate(prompt=PromptTemplate(
            template="You are a data annotator. Your task is to annotate a given example by scoring it with regards to several criteria.{context}",
            input_variables=[],
            partial_variables={
                "context": " The concrete context is the following: " + self.context if self.context and self.context != "" else ""
            }
        ))

        # Define the prompt template with the instructions to score an example along a set of features
        score_template = HumanMessagePromptTemplate(prompt=PromptTemplate(
            template="""\
##### Here is an example that you should annotate #####\n\n
{example}\n\n\n\
##### Instructions #####\n\n\
{instructions}\n\n\
{format_instructions}\n\n
Do not add any comments (starting with //) inside the JSON!""",
            input_variables=["example"],
            partial_variables={
                "instructions": "Score the above example along the following list of criteria. For each criteria, assign " + ("one of the possible values" if self.discrete_features else "an integer value from 0 to 10") + " that most accurately describes the example.",
                "format_instructions": output_parser.get_format_instructions()
            }
        ))

        # Assemble a list of chat messages
        messages = [
            system_template.format(),
            score_template.format(example=example, format_instructions=output_parser.get_format_instructions())
        ]

        # Execute the prompt and parse the output
        with get_openai_callback() as cb:
            try:
                output = llm(messages).content
                output = self._remove_json_comments(output)
                scores = output_parser.parse(output)
                used_fallback = False
            except Exception as e:
                # Report the exception to the callback for later analysis
                if self.callback:
                    self.callback.error_encountered(llm, messages[0].content, messages[1].content, output, e)

                # If the request is invalid (often because the LLM's context limit is exceeded), retry with the fallback model
                used_fallback = True
                print(f"Unexpected error occured using '{llm.model_name}': {str(e)}")
                scores = None
                if llm_fallback:
                    print(f"Retry using fallback model '{llm_fallback.model_name}'")
                    try:
                        output = llm_fallback(messages).content
                        output = self._remove_json_comments(output)
                        scores = output_parser.parse(output)
                    except Exception as e2:
                        # Report the exception to the callback for later analysis
                        if self.callback:
                            self.callback.error_encountered(llm_fallback, messages[0].content, messages[1].content, output, e)

                        print(f"Retry with fallback model '{llm_fallback.model_name}' unsuccessful: {str(e2)}")

            if scores:
                # Validate all scores (i.e., make sure that only allowed feature values are assigned)
                if self.discrete_features:
                    # Validate categorical features
                    for name, value in scores.items():
                        i = [f.name for f in feature_set.features].index(name)
                        allowed_values = feature_set.features[i].possible_values
                        # Replace all non-compliant values with 'NaN'
                        if value not in allowed_values:
                            scores[name] = np.nan
                            # print(f"Replaced non-compliant value '{value}' with 'NaN' for feature '{name}' with allowed values {allowed_values}.")
                else:
                    # Validate numeric feature values
                    for name, value in scores.items():
                        try:
                            numeric_value = float(value)                    # Try to convert to a float first (catches strings that can't be converted to numbers)
                            numeric_value = round(numeric_value)            # Now round to the nearest integer
                            if numeric_value < 0 or numeric_value > 10:
                                # If the value is not in the allowed range, throw an error
                                raise ValueError("Value must be in range 0-10!")
                        except ValueError:
                            # If a feature value is not compliant, replace it with 'NaN'
                            scores[name] = np.nan
            else:
                # If scoring failed, return a dictionary with empty scores
                scores = {}
                for feature in response_schemas:
                    scores[feature.name] = None

            # Report this LLM request to the callback for tracking and evaluation
            if self.callback:
                self.callback.example_transformed(llm_fallback if used_fallback else llm, example, feature_set, messages[0].content, messages[1].content, output, scores, cb.total_cost, cb.total_tokens, cb.prompt_tokens, cb.completion_tokens)

            return scores, cb


    def _remove_json_comments(self, json_str):
        # Remove all comments from JSON strings sometimes returned by GPT-4-Turbo as they often prevent parsing
        pattern = r'//.*?$'
        cleaned_json_str = re.sub(pattern, '', json_str, flags=re.MULTILINE)
        return cleaned_json_str

## Zero-Shot GPT Classifier

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted, check_array
from sklearn.utils.multiclass import unique_labels

from tqdm.notebook import tqdm

from datetime import datetime
import numpy as np


class PromptingTQDM(tqdm):
    def __init__(self, *args, **kwargs):
        super().__init__(bar_format="{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_inv_fmt}{postfix}]", *args, **kwargs)
        self.reset_stats()

    def reset_stats(self):
        # Initial stats
        self.tokens_in = 0
        self.tokens_out = 0
        self.costs = 0
        self.n_features = 0
        self.n_iter = 0

    def log_stats(self, tokens_in, tokens_out, cost, n_features):
        # Update statistics
        self.tokens_in += tokens_in
        self.tokens_out += tokens_out
        self.costs += cost
        self.n_features += n_features
        self.n_iter += 1

        # Calculate averages
        avg_tokens_in = self.tokens_in / self.n_iter
        avg_tokens_out = self.tokens_out / self.n_iter
        avg_cost = self.costs / self.n_iter

        # Update statistics in postfix string
        self.set_postfix_str(f"in {avg_tokens_in:.0f} tokens/{self.unit}, out {avg_tokens_out:.0f} tokens/{self.unit}, {avg_cost:.5f} USD/{self.unit}, {self.costs:.2f} USD total, {self.n_features:.0f} features total", refresh=True)



class ZeroShotGPTCallback:

    def __init__(self):
        self.consumption_log = []


    def log_consumption(self, llm, example: str, system_message: str, prompt_message: str, llm_output: str, total_cost: float, total_tokens: int, prompt_tokens: int, completion_tokens: int):
        self.consumption_log.append({
            "Timestamp": datetime.now().strftime("%Y-%m-%d, %H:%M:%S"),
            "LLM": llm.model_name,
            "Temperature": llm.temperature,
            "Example": example,
            "System Message": system_message,
            "Prompt Message": prompt_message,
            "LLM Output": llm_output,
            "Total Cost": total_cost,
            "Total Tokens": total_tokens,
            "Prompt Tokens": prompt_tokens,
            "Completion Tokens": completion_tokens
        })



class ZeroShotGPTClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self, context=None, llm_name="gpt-3.5-turbo", temperature=0.7, openai_api_key=None, openai_organization=None, callback: ZeroShotGPTCallback = None, verbose=False):
        self.context = context
        self.llm_name = llm_name
        self.temperature = temperature
        self.openai_api_key = openai_api_key
        self.openai_organization = openai_organization
        self.callback = callback
        self.verbose=verbose

        # Initialize the LLM for feature scoring
        self.llm = ChatOpenAI(model=self.llm_name, temperature=self.temperature, openai_api_key=self.openai_api_key, openai_organization=self.openai_organization)

        # Define the prompt template for the system message with the context and overall task instructions
        self.system_template = SystemMessagePromptTemplate(prompt=PromptTemplate(
            template="You are a data classifier. Your task is to classify a given example into one of multiple classes.{context}",
            input_variables=[],
            partial_variables={
                "context": " The concrete context is the following: " + self.context if self.context and self.context != "" else ""
            }
        ))

        # Define the prompt template with the instructions to classify an example
        self.classification_template = HumanMessagePromptTemplate(prompt=PromptTemplate(
            template="""\
##### Here is an example that you should classify #####\n\n
{example}\n\n\n\
##### Instructions #####\n\n\
{instructions} {classes}.""",
            input_variables=["example", "classes"],
            partial_variables={
                "instructions": "What is the most likely class of this example? Respond only with exactly one of the following class names and nothing else:"
            }
        ))


    def fit(self, X, y):
        # Store the classes and data seen during fit
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y

        # Return the classifier
        return self


    def predict(self, X):
        # Check if fit has been called
        check_is_fitted(self)

        # Use the data validation and preparation functionality implemented in FELIX
        temp_felix = FELIX()
        X, y = temp_felix._validate_data(X, None, check_y=False)

        # Classify each example in X
        predictions = []
        with PromptingTQDM(total=len(X), desc="Classifying examples", unit="prompt", disable=not self.verbose) as progress_bar:
            for i, example in X.items():
                with get_openai_callback() as cb:
                    # Assemble a list of chat messages
                    messages = [
                        self.system_template.format(),
                        self.classification_template.format(example=example, classes=", ".join([f"'{str(c)}'" for c in self.classes_]))
                    ]

                    # Execute the prompt and parse the output
                    prediction = self.llm(messages).content
                    if prediction in self.classes_:
                        predictions.append(prediction)
                    else:
                        print(f"Warning: prediction '{prediction}' is not a valid class.")
                        predictions.append(np.nan)

                    # Update the progress bar if self.verbose=True
                    if self.verbose:
                        progress_bar.log_stats(cb.prompt_tokens, cb.completion_tokens, cb.total_cost, 1)
                        progress_bar.update(1)

                    # Report this LLM request to the callback for tracking and evaluation
                    if self.callback and isinstance(self.callback, ZeroShotGPTCallback):
                        self.callback.log_consumption(self.llm, example, messages[0].content, messages[1].content, prediction, cb.total_cost, cb.total_tokens, cb.prompt_tokens, cb.completion_tokens)

        return predictions

## Fine-Tuned LLM

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.multiclass import unique_labels

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments, set_seed
import torch

import os
import random
import numpy as np
import pandas as pd


# Class which iteratively returns a model input string as a tensor
class TorchDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)



class FineTunedClassificationLLM(BaseEstimator, ClassifierMixin):

    def __init__(self, model_name="FacebookAI/roberta-base", max_tokens=512, epochs=1, model_path="./models/", output_path="./results", logging_path="./logs", seed=42):
        self.model_name = model_name
        self.max_tokens = max_tokens
        self.epochs = epochs
        self.model_path = model_path
        self.output_path = output_path
        self.logging_path = logging_path
        self.seed = seed

        self.training_args = TrainingArguments(
            seed = self.seed,                   # Random seed for initialization
            output_dir=self.output_path,        # Directory for storing model predictions and checkpoints
            logging_dir=self.logging_path,      # Directory for storing Tensorboard logs
            num_train_epochs=self.epochs,       # Number of training epochs to perform
            per_device_train_batch_size=16,     # Batch size per GPU/TPU core/CPU for training
            learning_rate = 5e-5,               # Initial learning rate for Adam
            fp16 = True,                        # Use 16-bit (mixed) precision training (through NVIDIA apex)
        )

        if not torch.cuda.is_available():
            print("Warning: Cuda is not available on this hardware. Training an LLM may run into problems.")

        # Set the seed for model initialization (for reproducible initial weights of the fully-connected classification layer)
        set_seed(self.seed)

        # Load model and tokenizer
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name, num_labels=2).to("cuda")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)


    def fit(self, X, y):
        # Remember the unique class labels
        self.classes_ = list(unique_labels(y))
        if len(self.classes_) != 2:
            raise ValueError(f"FineTunedClassificationLLM only supports binary classification but found {len(self.classes_)} unique classes.")

        # Map class labels to 0 and 1
        y = [self.classes_.index(label) for label in y]

        # Convert input data to list, if necessary (e.g., when data is provided as a Pandas Series)
        if not isinstance(X, list):
            X = list(X)

        # Tokenize the data
        train_encodings = self.tokenizer(X, padding=True, truncation=True, max_length=self.max_tokens)

        # Convert tokenized data into a torch Dataset
        train_dataset = TorchDataset(train_encodings, y)

        # Train the model
        trainer = Trainer(
            model=self.model,                    # the instantiated Transformers model to be trained
            args=self.training_args,             # training arguments which are defined above
            train_dataset=train_dataset,         # training dataset
        )
        trainer.train()

        # Save the model
        self.model.save_pretrained(self.model_path)
        self.tokenizer.save_pretrained(self.model_path)


    def predict(self, X):
        preds = []
        for x in X:
            # Tokenize the example
            inputs = self.tokenizer(x, padding=True, truncation=True, max_length=self.max_tokens, return_tensors="pt").to("cuda")

            # Perform inference with the model
            outputs = self.model(**inputs)

            # Convert into the predicted class label
            pred = self.classes_[outputs[0].softmax(1).argmax()]
            preds.append(pred)

        return preds

## Data Transformer

In [ ]:
# Imports for embedding transformation
from langchain.embeddings import OpenAIEmbeddings

# Imports for TF-IDF transformation
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize
import nltk

# Imports for imputation of missing values
from sklearn.impute import KNNImputer


class DataTransformer:

    def __init__(self, dataset: Dataset):
        self.dataset = dataset
        self.last_instance = None # Reference to the last trained model instance


    def get_raw(self):
        # Use transformation functionality already implemented in FELIX
        serializer = FELIX()

        # Convert multiple-column data into single-text data first
        X_train_raw = serializer._serialize_dataframe(self.dataset.X_train)
        X_test_raw = serializer._serialize_dataframe(self.dataset.X_test)

        # Name the Pandas Series
        X_train_raw.name = "Raw Text"
        X_test_raw.name = "Raw Text"

        self.last_instance = None

        return X_train_raw, X_test_raw


    def get_tfidf(self):
        # Convert multiple-column data into single-text data first
        X_train_raw, X_test_raw = self.get_raw()

        # Ensure the necessary NLTK data is downloaded
        nltk.download("punkt")
        nltk.download("stopwords")

        # Initialize stemmer
        stemmer = PorterStemmer()

        # Get the list of stop words from NLTK
        stop_words = set(stopwords.words("english"))

        # Custom tokenizer function using NLTK's word_tokenize, a stop words list, and a stemmer
        def tokenize_and_stem(text):
            tokens = [word.lower() for word in word_tokenize(text) if word.isalpha()]  # Remove non-alphabetic tokens
            filtered_tokens = [token for token in tokens if token not in stop_words]  # Remove stop words
            return [stemmer.stem(t) for t in filtered_tokens]  # Stemming

        # Initialize the vectorizer
        vectorizer = TfidfVectorizer(tokenizer=tokenize_and_stem)

        # Calculate the TF-IDF representation of the raw text
        X_train_tfidf = vectorizer.fit_transform(X_train_raw)
        X_test_tfidf = vectorizer.transform(X_test_raw)  # Note: use transform() not fit_transform() for test data
        self.last_instance = vectorizer

        # Transform sparse matrix into Pandas DataFrame
        X_train_tfidf = pd.DataFrame.sparse.from_spmatrix(X_train_tfidf, columns=vectorizer.get_feature_names_out())
        X_test_tfidf = pd.DataFrame.sparse.from_spmatrix(X_test_tfidf, columns=vectorizer.get_feature_names_out())

        return X_train_tfidf, X_test_tfidf


    def get_embeddings(self):
        # Convert multiple-column data into single-text data first
        X_train_raw, X_test_raw = self.get_raw()

        # Apply OpenAI embeddings
        embeddings = OpenAIEmbeddings()
        X_train_embeddings = np.array(embeddings.embed_documents(X_train_raw))
        X_test_embeddings = np.array(embeddings.embed_documents(X_test_raw))
        self.last_instance = embeddings

        # Convert to Pandas DataFrames
        X_train_embeddings = pd.DataFrame(X_train_embeddings, columns=[f"Dim {x}" for x in list(range(X_train_embeddings.shape[1]))])
        X_test_embeddings = pd.DataFrame(X_test_embeddings, columns=[f"Dim {x}" for x in list(range(X_test_embeddings.shape[1]))])

        return X_train_embeddings, X_test_embeddings


    def get_felix(self, discrete_features=False, gpt4=False, callback=None, verbose=True):
        # Use GPT-4, if requested, or GPT-3.5-Turbo (4K context variant by default and 16K context variant as fallback if requests exceed context window)
        llm_name = "gpt-4-1106-preview" if gpt4 else "gpt-3.5-turbo"
        llm_fallback = "gpt-4-1106-preview" if gpt4 else "gpt-3.5-turbo-16k"

        # Transform the data using FELIX
        self.last_instance = FELIX(context=self.dataset.context_train, llm_name=llm_name, llm_fallback=llm_fallback, discrete_features=discrete_features, zero_shot=self.dataset.train_zero_shot, callback=callback, verbose=verbose)
        X_train_felix = self.last_instance.fit_transform(self.dataset.X_train, self.dataset.y_train)
        self.last_instance.context = self.dataset.context_test
        X_test_felix = self.last_instance.transform(self.dataset.X_test)

        return X_train_felix, X_test_felix


    def one_hot_encode(self, X_train, X_test):
        # Concatenate the data
        X_train_test = pd.concat([X_train, X_test], axis=0)

        # Perform one-hot encoding
        X_train_test_onehot = pd.get_dummies(X_train_test)

        # Split the data back into train and test
        X_train_onehot = X_train_test_onehot.head(len(X_train))
        X_test_onehot = X_train_test_onehot.tail(len(X_test))

        return X_train_onehot, X_test_onehot


    def impute_missing_values(self, X_train, X_test, col_drop_thres=0.1, n_neighbors=5):
        def impute(df):
            # Remove all columns that have more than 'col_drop_thres' missing values
            missing_percentage = df.isnull().mean()                                         # Calculate the percentage of missing values for each column
            cols_to_drop = missing_percentage[missing_percentage > col_drop_thres].index    # Identify columns that have more than 'col_drop_thres' % missing values
            df = df.drop(columns=cols_to_drop)                                              # Drop the columns from the DataFrame

            # Estimate missing values with k-Nearest-Neighbors imputations
            imputer = KNNImputer(n_neighbors=n_neighbors, weights="distance")
            imputer.set_output(transform="pandas")
            df = imputer.fit_transform(df)

    	    # Round all values to integers
            df = df.round().astype(int)

            return df

        return impute(X_train), impute(X_test)


    def get_model_instance(self):
        # Return the last trained model instance
        return self.last_instance

## Run Tracking Functionality

Set your Weights & Biases project in lines 21 and 22 of the following cell:

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
from datetime import datetime
import time


class Run:

    def __init__(self, data_representation, classifier, dataset, seed=0):
        self.data_representation = [data_representation] if isinstance(data_representation, str) else data_representation
        self.classifier = [classifier] if isinstance(classifier, str) else classifier
        self.dataset = dataset
        self.seed = seed

        self.data_transformer = DataTransformer(dataset=d)
        self.cache = {}

        # Set Weights & Biases project reference for logging
        self.wandb_entity = "..."
        self.wandb_project = "..."


    def run(self):
        for dr in self.data_representation:
            for cls in self.classifier:
                self.single_run(dr, cls, self.dataset)


    def single_run(self, data_representation, classifier, dataset):
        print(f"Starting new run '{data_representation} {classifier}' on '{dataset.id}' ...")

        # Initiate logging of a new run
        self.init_logging(data_representation, classifier, dataset)

        # Prepare data representation
        if data_representation in self.cache:
            print(f"Loading cached data transformation '{data_representation}' ...")
            X_train_transformed, X_test_transformed, transformer_instance, callback, time_transformation = self.cache[data_representation]
        else:
            print(f"Transforming data using '{data_representation}' ...")
            start_time = time.time()
            X_train_transformed, X_test_transformed, transformer_instance, callback = self.transform_data(data_representation)
            time_transformation = time.time() - start_time
            self.cache[data_representation] = (X_train_transformed, X_test_transformed, transformer_instance, callback, time_transformation)

        print("X_train.shape =", X_train_transformed.shape)
        print("X_test.shape = ", X_test_transformed.shape)

        # Keep copies of the transformed data (before one-hot encoding and imputation) for logging
        X_train_transformed_original = X_train_transformed
        X_test_transformed_original = X_test_transformed

        # One-hot encode the data if is categorical
        if data_representation in ["FELIX GPT-3.5 (Categorical)", "FELIX GPT-4 (Categorical)"]:
            print("Creating one-hot encoding of categorical data ...")
            X_train_transformed, X_test_transformed = self.data_transformer.one_hot_encode(X_train_transformed, X_test_transformed)

        # Impute missing values if data is numerical
        elif data_representation in ["FELIX GPT-3.5 (Numerical)", "FELIX GPT-4 (Numerical)"]:
            print("Imputing missing values in numerical data ...")
            X_train_transformed, X_test_transformed = self.data_transformer.impute_missing_values(X_train_transformed, X_test_transformed)

        # Train the classifier on the training dataset and predict class labels on the test dataset
        start_time = time.time()
        print(f"Training '{classifier}' on '{dataset.id}' training set ...")
        self.classifier_instance = self.get_classifier(classifier)

        # If data was classified via ZeroShotGPTClassifier, extract its callback to log API cost and consumption
        if isinstance(self.classifier_instance, ZeroShotGPTClassifier):
            callback = self.classifier_instance.callback

        self.classifier_instance.fit(X_train_transformed, dataset.y_train)
        print(f"Predicting class labels of '{dataset.id}' test set with trained '{classifier}' ...")
        self.y_pred = self.classifier_instance.predict(X_test_transformed)
        time_classification = time.time() - start_time

        # For ZeroShotGPT classifiers, treat missing predictions as incorrect answers
        if isinstance(self.classifier_instance, ZeroShotGPTClassifier):
            # Find the unique ground truth labels
            unique_labels = set(self.dataset.y_test)

            # Treat missing predictions as incorrect answers by setting them to the incorrect ground truth class
            counter = 0
            for i in range(len(self.y_pred)):
                if self.y_pred[i] not in unique_labels:
                    # Find the incorrect label (the one not equal to the ground truth) and assign it as the prediction
                    self.y_pred[i] = (unique_labels - {self.dataset.y_test.to_list()[i]}).pop()
                    counter += 1

            if counter > 0:
                print(f"Treating {counter} missing values as incorrect answers.")

        # Log the final results and finish the run
        print("Finishing run and logging the results ...")
        self.log(self.y_pred, X_train_transformed_original, X_test_transformed_original, self.classifier_instance, transformer_instance, callback, time_transformation, time_classification)
        self.finish()


    def transform_data(self, data_representation):
        if data_representation == "TF-IDF":
            callback = None
            X_train_transformed, X_test_transformed = self.data_transformer.get_tfidf()
        elif data_representation == "Embeddings":
            callback = None
            X_train_transformed, X_test_transformed = self.data_transformer.get_embeddings()
        elif data_representation == "Raw Text":
            callback = None
            X_train_transformed, X_test_transformed = self.data_transformer.get_raw()
        elif data_representation == "FELIX GPT-3.5 (Numerical)":
            callback = CustomFELIXCallback()
            X_train_transformed, X_test_transformed = self.data_transformer.get_felix(discrete_features=False, gpt4=False, callback=callback, verbose=True)
        elif data_representation == "FELIX GPT-3.5 (Categorical)":
            callback = CustomFELIXCallback()
            X_train_transformed, X_test_transformed = self.data_transformer.get_felix(discrete_features=True, gpt4=False, callback=callback, verbose=True)
        elif data_representation == "FELIX GPT-4 (Numerical)":
            callback = CustomFELIXCallback()
            X_train_transformed, X_test_transformed = self.data_transformer.get_felix(discrete_features=False, gpt4=True, callback=callback, verbose=True)
        elif data_representation == "FELIX GPT-4 (Categorical)":
            callback = CustomFELIXCallback()
            X_train_transformed, X_test_transformed = self.data_transformer.get_felix(discrete_features=True, gpt4=True, callback=callback, verbose=True)
        else:
            raise ValueError(f"Invalid value '{data_representation}' for data representation. Must be one of ['TF-IDF', 'Embeddings', 'Raw Text', 'FELIX GPT-3.5 (Numerical)', 'FELIX GPT-3.5 (Categorical)', 'FELIX GPT-4 (Numerical)', 'FELIX GPT-4 (Categorical)'].")

        transformer_instance = self.data_transformer.get_model_instance()

        return X_train_transformed, X_test_transformed, transformer_instance, callback


    def get_classifier(self, classifier):
        if classifier == "RandomForest":
            return RandomForestClassifier(random_state=self.seed)
        elif classifier == "LogisticRegression":
            return LogisticRegression(max_iter=10000, random_state=seed)
        elif classifier == "GPT-3.5":
            return ZeroShotGPTClassifier(llm_name="gpt-3.5-turbo", callback=ZeroShotGPTCallback(), verbose=True)
        elif classifier == "GPT-4":
            return ZeroShotGPTClassifier(llm_name="gpt-4-1106-preview", callback=ZeroShotGPTCallback(), verbose=True)
        elif classifier == "RoBERTA-Base 1 Epoch":
            return FineTunedClassificationLLM(model_name="FacebookAI/roberta-base", epochs=1, seed=self.seed)
        elif classifier == "RoBERTA-Base 10 Epochs":
            return FineTunedClassificationLLM(model_name="FacebookAI/roberta-base", epochs=10, seed=self.seed)
        elif classifier == "RoBERTA-Base 50 Epochs":
            return FineTunedClassificationLLM(model_name="FacebookAI/roberta-base", epochs=50, seed=self.seed)
        elif classifier == "RoBERTA-Base 100 Epochs":
            return FineTunedClassificationLLM(model_name="FacebookAI/roberta-base", epochs=100, seed=self.seed)

        raise ValueError(f"Invalid value '{classifier}' for classifier. Must be one of ['RandomForest', 'LogisticRegression', 'GPT-3.5', 'GPT-4'].")


    def init_logging(self, data_representation, classifier, dataset):
        # Start a new Weights & Biases run to track results
        wandb.init(
            entity=self.wandb_entity,
            project=self.wandb_project,
            name=f"{data_representation} {classifier}",
            config={
                "dataset": dataset.id,
                "dataset short name": dataset.short_name,
                "data_representation": data_representation,
                "model": classifier,
                "context_description": dataset.context_train,
                "context_description_test": dataset.context_test,
                "n_train": dataset.X_train.shape[0],
                "n_test": dataset.X_test.shape[0],
                "seed": self.seed,
                "pos_class": dataset.pos_class,
                "train_zero_shot": dataset.train_zero_shot
            }
        )


    def log(self, y_pred, X_train_transformed, X_test_transformed, classifier=None, transformer=None, callback=None, time_transformation=-1.0, time_classification=-1.0):
        # Log classification performance scores
        results = self.calculate_scores(self.dataset.y_test, y_pred, pos_class=self.dataset.pos_class)

        # Store the run duration
        results["Time Transformation"] = time_transformation
        results["Time Classification"] = time_classification
        results["Time Total"] = time_transformation + time_classification

        # Calculate API costs
        if callback and isinstance(callback, FELIXCallback):
            generation_cost = sum([x["Total Cost"] for x in callback.generation_log])
            scoring_cost = sum([x["Total Cost"] for x in callback.scoring_log])
            total_cost = generation_cost + scoring_cost
            results["Generation Cost"] = generation_cost
            results["Scoring Cost"] = scoring_cost
            results["Total Cost"] = total_cost
        elif callback and isinstance(callback, ZeroShotGPTCallback):
            results["Total Cost"] = sum([x["Total Cost"] for x in callback.consumption_log])

        # Save a detailed log of all LLM requests
        if callback and isinstance(callback, FELIXCallback):
            results["Generation Log"] = wandb.Table(dataframe=pd.DataFrame(callback.generation_log))
            results["Scoring Log"] = wandb.Table(dataframe=pd.DataFrame(callback.scoring_log))
        elif callback and isinstance(callback, ZeroShotGPTCallback):
            results["Prompt Log"] = wandb.Table(dataframe=pd.DataFrame(callback.consumption_log))

        # Store feature insights from classifier models
        if classifier:
            # If features have been learned by FELIX, get its features descriptions
            if transformer and isinstance(transformer, FELIX):
                df_features = transformer.get_features_as_dataframe().rename(columns={"Name": "Feature Name"})
                if "Possible Values" in df_features.columns:
                    df_features["Possible Values"] = df_features["Possible Values"].apply(lambda x: str(x).strip("[]").replace(", ", "\n"))
            else:
                df_features = None

            if isinstance(classifier, RandomForestClassifier):
                df_importances = pd.DataFrame({"Feature Name": classifier.feature_names_in_, "Importance": classifier.feature_importances_}).sort_values(by="Importance", ascending=False)  # Get feature importance scores from the Random Forest classifier
                if transformer and isinstance(transformer, FELIX) and transformer.discrete_features:
                    df_importances[["Feature Name", "Value"]] = df_importances["Feature Name"].str.rsplit("_", n=1, expand=True)                                                            # Separate feature names and values from the one-hot encoding
                if isinstance(df_features, pd.DataFrame):
                    df_importances = df_importances.set_index("Feature Name").join(df_features.set_index("Feature Name"), how="left").reset_index()                                         # Join the Random Forest feature importances with FELIX's feature descriptions
                results["Feature Importance"] = df_importances
            elif isinstance(classifier, LogisticRegression):
                df_coefficients = pd.DataFrame({"Feature Name": classifier.feature_names_in_, "Coefficient": classifier.coef_[0]}).sort_values(by="Coefficient", ascending=False)           # Get coefficients from the Logistic Regression classifier
                if transformer and isinstance(transformer, FELIX) and transformer.discrete_features:
                    df_coefficients[["Feature Name", "Value"]] = df_coefficients["Feature Name"].str.rsplit("_", n=1, expand=True)                                                            # Separate feature names and values from the one-hot encoding
                if isinstance(df_features, pd.DataFrame):
                    df_coefficients = df_coefficients.set_index("Feature Name").join(df_features.set_index("Feature Name"), how="left").reset_index()                                       # Join the Logistic Regression coefficients with FELIX's feature descriptions
                results["Coefficients"] = df_coefficients

        # Store the trained FELIX instance, if available
        if transformer:
            if isinstance(transformer, FELIX):
                filename = f"FELIX-{datetime.now().strftime('%Y-%m-%d-%H:%M:%S')}.json"
                transformer.save_instance(filename)
                wandb.save(filename)

        wandb.log(results)

        # Log training and test data
        try:
            wandb.log({
                "Training Data": wandb.Table(dataframe=pd.concat([pd.Series(self.dataset.y_train, name="Ground Truth").reset_index(drop=True), self.dataset.X_train.reset_index(drop=True), X_train_transformed.reset_index(drop=True)], axis=1)),
                "Test Data": wandb.Table(dataframe=pd.concat([pd.Series(self.dataset.y_test, name="Ground Truth").reset_index(drop=True), pd.Series(y_pred, name="Prediction").reset_index(drop=True), self.dataset.X_test.reset_index(drop=True), X_test_transformed.reset_index(drop=True)], axis=1))
            })
        except TypeError as e:
            # Some formats of data seem to cause an internal TypeError within Weights & Biases that I could not solve yet
            print(e)

        return results


    def calculate_scores(self, y_true, y_pred, pos_class):
        return {
            "Accuracy": accuracy_score(y_true, y_pred),
            "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred),
            "F1 Score (Macro)": f1_score(y_true, y_pred, average="macro"),
            "F1 Score (Positive Class)": f1_score(y_true, y_pred, pos_label=pos_class),
            "Precision (Macro)": precision_score(y_true, y_pred, average="macro"),
            "Precision (Positive Class)": precision_score(y_true, y_pred, pos_label=pos_class),
            "Recall (Macro)": recall_score(y_true, y_pred, average="macro"),
            "Recall (Positive Class)": recall_score(y_true, y_pred, pos_label=pos_class)
        }


    def finish(self):
        wandb.finish()



class CustomFELIXCallback(FELIXCallback):

    def __init__(self):
        self.generation_log = []
        self.scoring_log = []
        self.error_log = []


    def features_generated_for_pair(self, llm, example_a: str, example_b: str, label_a: str, label_b: str, system_message: str, prompt_message: str, llm_output: str, features: NumericalFeatureSet | CategoricalFeatureSet, total_cost, total_tokens, prompt_tokens, completion_tokens):
        self.generation_log.append({
            "Timestamp": datetime.now().strftime("%Y-%m-%d, %H:%M:%S"),
            "LLM": llm.model_name,
            "Temperature": llm.temperature,
            "Example A": example_a,
            "Label A": label_a,
            "Example B": example_b,
            "Label B": label_b,
            "System Message": system_message,
            "Prompt Message": prompt_message,
            "LLM Output": llm_output,
            "Features": str(features.features),
            "Total Cost": total_cost,
            "Total Tokens": total_tokens,
            "Prompt Tokens": prompt_tokens,
            "Completion Tokens": completion_tokens
        })


    def example_transformed(self, llm, example: str, feature_set: NumericalFeatureSet | CategoricalFeatureSet, system_message: str, prompt_message: str, llm_output: str, scores, total_cost: float, total_tokens: int, prompt_tokens: int, completion_tokens: int):
        self.scoring_log.append({
            "Timestamp": datetime.now().strftime("%Y-%m-%d, %H:%M:%S"),
            "LLM": llm.model_name,
            "Temperature": llm.temperature,
            "Example": example,
            "Features": str(feature_set.features),
            "System Message": system_message,
            "Prompt Message": prompt_message,
            "LLM Output": llm_output,
            "Scores": str(scores),
            "Total Cost": total_cost,
            "Total Tokens": total_tokens,
            "Prompt Tokens": prompt_tokens,
            "Completion Tokens": completion_tokens
        })


    def error_encountered(self, llm, system_message, prompt_message, llm_output, error):
        self.error_log.append({
            "Timestamp": datetime.now().strftime("%Y-%m-%d, %H:%M:%S"),
            "LLM": llm,
            "System Message": system_message,
            "Prompt Message": prompt_message,
            "LLM Output": llm_output,
            "Error": error
        })

# Experiment 1: Overall Performance

In [ ]:
d = dataset_list["reviews-amazon"]
d

## TF-IDF

In [ ]:
run = Run("TF-IDF", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## Text Embeddings

In [ ]:
run = Run("Embeddings", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## GPT-3.5 Zero-Shot

In [ ]:
run = Run("Raw Text", "GPT-3.5", d, seed)
run.run()

## GPT-4 Zero-Shot

In [ ]:
run = Run("Raw Text", "GPT-4", d, seed)
run.run()

## FELIX GPT-3.5 (Numerical)

In [ ]:
run = Run("FELIX GPT-3.5 (Numerical)", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## FELIX GPT-3.5 (Categorical)

In [ ]:
run = Run("FELIX GPT-3.5 (Categorical)", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## FELIX GPT-4 (Numerical)

In [ ]:
run = Run("FELIX GPT-4 (Numerical)", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## FELIX GPT-4 (Categorical)

In [ ]:
run = Run("FELIX GPT-4 (Categorical)", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## Fine-Tuned LLM

In [ ]:
run = Run("Raw Text", "RoBERTA-Base 100 Epochs", d, seed + i)
run.run()

# Experiment 2: Sample Efficiency

In [ ]:
d = dataset_list["sentiment"]
n_train = 10

d = Dataset(
    id=d.id,
    short_name=d.short_name,
    X_train=d.X_train.head(n_train),
    X_test=d.X_test,
    y_train=d.y_train.head(n_train),
    y_test=d.y_test,
    pos_class=d.pos_class,
    context_train=d.context_train,
    context_test=d.context_test,
    train_zero_shot=False
)

d

## TF-IDF

In [ ]:
run = Run("TF-IDF", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## Text Embeddings

In [ ]:
run = Run("Embeddings", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## FELIX GPT-3.5 (Numerical)

In [ ]:
run = Run("FELIX GPT-3.5 (Numerical)", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## FELIX GPT-3.5 (Categorical)

In [ ]:
run = Run("FELIX GPT-3.5 (Categorical)", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## FELIX GPT-4 (Numerical)

In [ ]:
run = Run("FELIX GPT-4 (Numerical)", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## FELIX GPT-4 (Categorical)

In [ ]:
run = Run("FELIX GPT-4 (Categorical)", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## Fine-Tuned LLM

In [ ]:
run = Run("Raw Text", "RoBERTA-Base 100 Epochs", d_sample, seed + i)
run.run()

# Experiment 3: Domain Adaptation

In [ ]:
train_name = "reviews-amazon"
test_name = "reviews-yelp"

d_train = dataset_list[train_name]
d_test = dataset_list[test_name]

if d_train.pos_class != d_test.pos_class:
    print("Warning: Train and test datasets have different classes. Do you still want to continue?\n")

d = Dataset(
    id=f"{d_train.id}>{d_test.id}",
    short_name=f"{d_train.short_name}>{d_test.short_name}",
    X_train=d_train.X_train,
    X_test=d_test.X_test,
    y_train=d_train.y_train,
    y_test=d_test.y_test,
    pos_class=d_train.pos_class,
    context_train=d_train.context_train,
    context_test=d_test.context_test,
    train_zero_shot=False
)

d

## TF-IDF

In [ ]:
run = Run("TF-IDF", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## Text Embeddings

In [ ]:
run = Run("Embeddings", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## FELIX GPT-3.5 (Numerical)

In [ ]:
run = Run("FELIX GPT-3.5 (Numerical)", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## FELIX GPT-3.5 (Categorical)

In [ ]:
run = Run("FELIX GPT-3.5 (Categorical)", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## FELIX GPT-4 (Numerical)

In [ ]:
run = Run("FELIX GPT-4 (Numerical)", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## FELIX GPT-4 (Categorical)

In [ ]:
run = Run("FELIX GPT-4 (Categorical)", ["RandomForest", "LogisticRegression"], d, seed)
run.run()

## Fine-Tuned LLM

In [ ]:
run = Run("Raw Text", "RoBERTA-Base 100 Epochs", d, seed + i)
run.run()

# Experiment 5: Internal Validity

## Experiment 5.1: Clustering Validity

### Learn Numeric Feature Set

In [ ]:
results_log = []

In [ ]:
def log_results(dataset_id, method, felix, features, f1_rf, f1_lr, probs_lr, comment=""):
    results_log.append({
        "dataset": dataset_id,
        "method": method,
        "felix_variant": f"FELIX GPT-{'3.5' if '3.5' in felix.llm_scoring else '4'} {'Categorical' if felix.discrete_features else 'Numerical'}",
        "n_features": len(features),
        "feature_set": features,
        "f1_rf": f1_rf,
        "f1_lr": f1_lr,
        "probs_lr": probs_lr,
        "comment": comment
    })

In [ ]:
# Select a dataset
d = dataset_list["sentiment"]
d

In [ ]:
# Configure FELIX
felix = FELIX(
    context=d.context_train,
    temperature_scoring=0.0,
    llm_scoring="gpt-3.5-turbo-16k",
    discrete_features=False,
    verbose=True
)

In [ ]:
n_generation = 30

# Fit FELIX to the dataset to learn features
felix._full_feature_set = felix.generate_features(d.X_train.head(n_generation), d.y_train.head(n_generation))
felix._full_feature_set = felix._ensure_unique_feature_names(felix._full_feature_set)
felix._features = felix._full_feature_set

Save the learned features for later use:

In [ ]:
from google.colab import files
import datetime

# Serializing json
json_object = felix._features.json(indent=4)

# Store the results
filename = f"{datetime.date.today().strftime('%Y_%m_%d')} - FELIX Features {d.short_name}.json"
with open(filename, "w") as f:
    f.write(json_object)

# Download the results
files.download(filename)

In [ ]:
from google.colab import files
import json

# Upload the features JSON
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Parse the results
flat_list = json.loads(uploaded[filename])["features"]
if felix.discrete_features:
    felix._features = CategoricalFeatureSet(features=flat_list)
else:
    felix._features = NumericalFeatureSet(features=flat_list)
felix._full_feature_set = felix._features
len(felix._features.features)

### Score All Learned Features

In [ ]:
df_scores = felix.transform(d.X_test)
df_scores

In [ ]:
from google.colab import files
import datetime

# Save the scores data as a CSV
filename = f"{datetime.date.today().strftime('%Y_%m_%d')} - FELIX Scores {d.short_name}.csv"
df_scores.to_csv(filename, index=False)

# Download the results
files.download(filename)

In [ ]:
from google.colab import files
import datetime

# Upload the scores CSV
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Load the scores
df_scores = pd.read_csv(filename)
df_scores

### Handle Missing Values (Numerical)

In [ ]:
col_drop_thres = 0.1
n_neighbors = 5

# Remove all columns that have more than 'col_drop_thres' missing values
missing_percentage = df_scores.isnull().mean()                                  # Calculate the percentage of missing values for each column
cols_to_drop = missing_percentage[missing_percentage > col_drop_thres].index    # Identify columns that have more than 'col_drop_thres' % missing values
df_scores = df_scores.drop(columns=cols_to_drop)                                # Drop the columns from the DataFrame

# Estimate missing values with k-Nearest-Neighbors imputations
imputer = KNNImputer(n_neighbors=n_neighbors, weights="distance")
imputer.set_output(transform="pandas")
df_scores = imputer.fit_transform(df_scores)

print("Columns removed:", cols_to_drop)
print("Columns with missing features remaining:", df_scores.isnull().any().sum())

In [ ]:
# Remove features from FELIX that could not be scored reliably (i.e., more than 'col_drop_thres' missing values)
felix._features.features = [f for f in felix._features.features if f.name not in cols_to_drop]

print(len(felix._features.features))
print(df_scores.shape)

### Handle Missing Values (Categorical)

In [ ]:
nan_columns = df_scores.isna().all()[df_scores.isna().all() == True].index.to_list()

print(nan_columns)

In [ ]:
# Remove features from FELIX that could not be scored (i.e., all rows have NaN values)
felix._features.features = [f for f in felix._features.features if f.name not in nan_columns]

# Remove respective columns from the scores dataset
df_scores = df_scores.drop(columns=nan_columns)

print(len(felix._features.features))
print(df_scores.shape)

### Create Feature Embeddings

In [ ]:
# Create text embeddings for each feature
felix._feature_embeddings = felix._create_feature_embeddings(felix._features)

### Prepare Feature Selection Code

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


def encode_data(X_train, X_test):
    # Drop columns that have no values
    nan_columns_train = X_train.isna().all()[X_train.isna().all() == True].index.to_list()
    nan_columns_test = X_test.isna().all()[X_test.isna().all() == True].index.to_list()
    # nan_columns = nan_columns_train + nan_columns_test
    nan_columns = [x for x in nan_columns_train if x in nan_columns_test]
    X_train = X_train.drop(columns=nan_columns)
    X_test = X_test.drop(columns=nan_columns)

    # Identify categorical columns (assuming the same columns in train and test)
    categorical_columns = X_train.select_dtypes(include=[object]).columns
    non_categorical_columns = X_train.select_dtypes(exclude=[object]).columns

    # Encode values based on column type
    if len(categorical_columns) > 0 and len(non_categorical_columns) == 0:
        data_transformer = DataTransformer(dataset=None)
        return data_transformer.one_hot_encode(X_train, X_test)
    elif len(non_categorical_columns) > 0 and len(categorical_columns) == 0:
        return X_train, X_test
    else:
        print("Categorical columns =", categorical_columns)
        print("Numerical columns =", non_categorical_columns)
        print("NaN columns training =", nan_columns_train)
        print("NaN columns test =", nan_columns_test)
        print("NaN columns =", nan_columns)
        raise ValueError("Cannot handle mixed dataset of categorical and numerical columns.")


def evaluate_feature_set(feature_set, X, y, cv_splits=5, random_state=42, lr_probs=True, lr_f1=True, rf_f1=True):
    # Define classifier models
    if lr_probs or lr_f1:
        lr_model = LogisticRegression(max_iter=10000, random_state=random_state)
    if rf_f1:
        rf_model = RandomForestClassifier(random_state=random_state)

    # Create lists to store the scores from each cross-validation fold
    avg_probs_lr = []
    avg_scores_lr = []
    avg_scores_rf = []

    # Evaluate performance using k-fold cross validation
    kf = KFold(n_splits=cv_splits)
    for train_index, test_index in kf.split(X):
        # Create train-test split
        X_train = X.iloc[train_index][feature_set]
        y_train = y.iloc[train_index]
        X_test = X.iloc[test_index][feature_set]
        y_test = y.iloc[test_index]

        # Convert columns that contain categorical data into a one-hot representation
        X_train, X_test = encode_data(X_train, X_test)

        # Train the models with the feature set
        if lr_probs or lr_f1:
            lr_model.fit(X_train, y_train)
        if rf_f1:
            rf_model.fit(X_train, y_train)

        # Evaluate the trained models on the test set
        if lr_probs:
            probs_lr = lr_model.predict_proba(X_test)
            avg_prob = np.mean([p[lr_model.classes_.tolist().index(c)] for p, c in zip(probs_lr, y_test)]) # Calculate the average predicted probability of belonging to the ground truth class
            avg_probs_lr.append(avg_prob)
        else:
            avg_probs_lr.append(0.0)
        if lr_f1:
            preds_lr = lr_model.predict(X_test)
            f1_lr = f1_score(y_test, preds_lr, average="macro")
            avg_scores_lr.append(f1_lr)
        else:
            avg_scores_lr.append(0.0)
        if rf_f1:
            preds_rf = rf_model.predict(X_test)
            f1_rf = f1_score(y_test, preds_rf, average="macro")
            avg_scores_rf.append(f1_rf)
        else:
            avg_scores_rf.append(0.0)

    # Calculate and return average scores across all folds
    return np.mean(avg_probs_lr), np.mean(avg_scores_lr), np.mean(avg_scores_rf)


def forward_selection(X, y, cv_splits=5, random_state=42, lr_probs=True, lr_f1=True, rf_f1=True):
    # Create lists to store the features selected (in order of forward selection) and corresponding scores
    feature_set = []
    scores = []

    # Iteratively add the feature that leads to the best classification performance
    while len(feature_set) < X.shape[1]:
        # Keep track of the best performance so far
        best_feature = None
        best_score = -np.inf
        prob_lr = -np.inf
        score_lr = -np.inf
        score_rf = -np.inf

        # Iterate over all features that are not yet in the feature set and find the one that leads to the best performance
        for feature in X.columns:
            if feature not in feature_set:
                # Add the current feature to the set
                features_to_use = feature_set + [feature]

                # Evaluate the classification performance using this feature set
                lr_prob, lr_score, rf_score = evaluate_feature_set(features_to_use, X, y, cv_splits, random_state, lr_probs=lr_probs, lr_f1=lr_f1, rf_f1=rf_f1)

                # Calculate the final decision criterion
                if lr_f1 and rf_f1:
                    score = np.mean([lr_score, rf_score])
                elif lr_f1:
                    score = lr_score
                elif rf_f1:
                    score = rf_score
                elif lr_probs:
                    score = lr_prob

                # Check if this is the best score
                if score > best_score:
                    best_feature = feature
                    best_score = score

                    # Also store the other scores as reference
                    prob_lr = lr_prob
                    score_lr = lr_score
                    score_rf = rf_score

        # Add best feature to our feature set
        feature_set.append(best_feature)
        scores.append(best_score)

        log_results(d.id, "Supervised Forward Selection", felix, feature_set, score_rf, score_lr, prob_lr, f"Added feature {best_feature}")

        print(f"n_features = {len(feature_set)}. Score = {best_score}. LR_prob = {prob_lr}. LR_F1 = {score_lr}. RF_F1 = {score_rf}. Added feature {best_feature}")

    return feature_set, scores

### Supervised Forward Selection

In [ ]:
features_forward, scores_forward = forward_selection(df_scores, d.y_test, cv_splits=5, random_state=seed, lr_probs=True, lr_f1=True, rf_f1=True)

In [ ]:
from google.colab import files

df_forward = pd.DataFrame({"Feature": features_forward, "F1 (LR-RF avg.)": scores_forward})

filename = "2023_11_02 - FELIX Forward Selection Fake News.csv"
df_forward.to_csv(filename, index=False)

# Download the results
files.download(filename)

In [ ]:
# Upload the results
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Load the scores
df_forward = pd.read_csv(filename)

features_forward = df_forward["Feature"].tolist()
scores_forward = df_forward["F1 (LR-RF avg.)"].tolist()

### HDBSCAN

In [ ]:
def run_hdbscan(feature_embeddings, df_scores, d, keep_noise=True, cluster_selection_method="leaf", cluster_selection_epsilon=0.0, alpha=1.0, verbose=False):
    # Cluster the features with HDBSCAN
    np.random.seed(0)
    hdbscan = HDBSCAN(min_cluster_size=2, allow_single_cluster=True, cluster_selection_method=cluster_selection_method, cluster_selection_epsilon=cluster_selection_epsilon, alpha=alpha)
    cluster_labels = hdbscan.fit_predict(feature_embeddings)

    if keep_noise:
        # Assign all features identified as noise (i.e., not part of a cluster) to their own cluster
        next_label = np.max(cluster_labels) + 1
        for i in range(len(cluster_labels)):
            if cluster_labels[i] == -1:
                cluster_labels[i] = next_label
                next_label += 1

        # Select the most representative feature from each cluster
        features_hdbscan = felix._select_representative_features(felix._features, felix._feature_embeddings, cluster_labels)
        if verbose:
            print(f"HDBSCAN selected {len(features_hdbscan.features)} features (including noise)")
    else:
        # If all features have been identified as noise, put them all in one cluster
        if (np.array(cluster_labels) == -1).all():
            cluster_labels = [0 for _ in cluster_labels]
            if verbose:
                print("All features identified as noise. Treating them as one cluster")

        # Prune the feature set to only those features that are not identified as noise
        pruned_feature_set = felix._features.copy(deep=True)
        pruned_feature_set.features = [f for f, l in zip(felix._features.features, cluster_labels) if l != -1]
        pruned_embeddings = np.array([e for e, l in zip(felix._feature_embeddings, cluster_labels) if l != -1])
        pruned_labels = [l for l in cluster_labels if l != -1]

        # Select the most representative feature from each cluster (excluding noise)
        features_hdbscan = felix._select_representative_features(pruned_feature_set, pruned_embeddings, pruned_labels)
        if verbose:
            print(f"HDBSCAN selected {len(features_hdbscan.features)} features (without noise)")


    lr_prob, lr_score, rf_score = evaluate_feature_set([f.name for f in features_hdbscan.features], df_scores, d.y_test, cv_splits=5, random_state=42, lr_probs=True, lr_f1=True, rf_f1=True)
    score_hdbscan = np.mean([lr_score, rf_score])

    return features_hdbscan, rf_score, lr_score, lr_prob

In [ ]:
features_hdbscan, rf_score, lr_score, lr_prob = run_hdbscan(felix._feature_embeddings, df_scores, d, keep_noise=True, cluster_selection_method="leaf", cluster_selection_epsilon=0.0, alpha=1.0)
log_results(d.id, "HDBSCAN (w/ noise)", felix, [f.name for f in features_hdbscan.features], rf_score, lr_score, lr_prob, comment="keep_noise=True, cluster_selection_method='leaf', cluster_selection_epsilon=0.0, alpha=1.0")
print(f"F1 = {np.mean([rf_score, lr_score])}. n_features = {len(features_hdbscan.features)}")

features_hdbscan, rf_score, lr_score, lr_prob = run_hdbscan(felix._feature_embeddings, df_scores, d, keep_noise=False, cluster_selection_method="leaf", cluster_selection_epsilon=0.0, alpha=1.0)
log_results(d.id, "HDBSCAN (w/o noise)", felix, [f.name for f in features_hdbscan.features], rf_score, lr_score, lr_prob, comment="keep_noise=False, cluster_selection_method='leaf', cluster_selection_epsilon=0.0, alpha=1.0")
print(f"F1 = {np.mean([rf_score, lr_score])}. n_features = {len(features_hdbscan.features)}")

Perform a grid search for the optimal HDBSCAN hyperparameters

In [ ]:
options_keep_noise = [True, False]
options_cluster_selection_method = ["leaf", "eom"]
options_cluster_selection_epsilon = [float(x) / 10 for x in range(0, 11, 2)]
options_alpha = [float(x) / 10 for x in range(5, 16, 1)]

tuning_results = []

In [ ]:
for kn in options_keep_noise:
    for csm in options_cluster_selection_method:
        for epsilon in options_cluster_selection_epsilon:
            for alpha in options_alpha:
                features_hdbscan, rf_score, lr_score, lr_prob = run_hdbscan(felix._feature_embeddings, df_scores, d, keep_noise=kn, cluster_selection_method=csm, cluster_selection_epsilon=epsilon, alpha=alpha)
                r = {
                    "Dataset": d.id,
                    "keep_noise": kn,
                    "cluster_selection_method": csm,
                    "epsilon": epsilon,
                    "alpha": alpha,
                    "F1 Average": np.mean([rf_score, lr_score]),
                    "F1 LR": lr_score,
                    "F1 RF": rf_score,
                    "n_features": len(features_hdbscan.features)
                }
                tuning_results.append(r)
                print(r)

Download/upload the results of hyperparameter optimization

In [ ]:
df_tuning_results = pd.DataFrame(tuning_results)
df_tuning_results

In [ ]:
from google.colab import files
import datetime

# Save the scores data as a CSV
filename = f"{datetime.date.today().strftime('%Y_%m_%d')} - HDBSCAN Hyperparameter Optimization.csv"
df_tuning_results.to_csv(filename, index=False)

# Download the results
files.download(filename)

In [ ]:
from google.colab import files
import datetime

# Upload the scores CSV
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Load the scores
df_tuning_results = pd.read_csv(filename)
df_tuning_results

Plot the results of hyperparameter optimization

In [ ]:
# Simplify the dataset IDs for visualization
df_tuning_results["Dataset"] = df_tuning_results["Dataset"].map({
    "cardiffnlp/tweet_sentiment_multilingual": "Sentiment",
    "hate_speech18": "Hate Speech",
    "amazon_polarity": "Amazon",
    "GonzaloA/fake_news": "Fake News",
    "tum-nlp/IDMGSP": "Papers"
})

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
from google.colab import files

# Set up the matplotlib figure
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Unpack the axis for easy reference
(ax1, ax2) = axs

# Plot boxplot for keep_noise
sns.boxplot(x='Dataset', y='F1 Average', hue='keep_noise', data=df_tuning_results, ax=ax1)
ax1.set_title("Performance for Different Values of 'keep_noise'")
ax1.set_ylabel("F1 Score")
ax1.set_xlabel("")
ax1.set_ylim([0.3, 1.0])
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# Plotting boxplot for cluster_selection_method
sns.boxplot(x='Dataset', y='F1 Average', hue='cluster_selection_method', data=df_tuning_results, ax=ax2)
ax2.set_title("Performance for Different Values of 'cluster_selection_method'")
ax2.set_ylabel("F1 Score")
ax2.set_xlabel("")
ax2.set_ylim([0.3, 1.0])
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# Add an overall title
fig.suptitle("HDBSCAN: Hyperparameter Tuning (Categorical Features)")

# Adjust the layout
plt.tight_layout()

# Download the plot as PDF
filename = "HDBSCAN Categorical Hyperparameter Optimization A.pdf"
plt.savefig(filename)
files.download(filename)

# Display the plots
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


# Filter the dataframe for the optimal values of other hyperparameters
df_filtered = df_tuning_results[
    (df_tuning_results['keep_noise'] == False) &
    (df_tuning_results['cluster_selection_method'] == 'leaf')
]

# Get unique datasets
unique_datasets = df_filtered['Dataset'].unique()

# Prepare contour plots data with the filtered dataframe
contour_plots_data = []
for dataset in unique_datasets:
    dataset_df = df_filtered[df_filtered['Dataset'] == dataset]
    dataset_grouped = dataset_df.groupby(['alpha', 'epsilon'])['F1 Average'].mean().reset_index()
    pivot_table = dataset_grouped.pivot(index='alpha', columns='epsilon', values='F1 Average')
    contour_plots_data.append((dataset, pivot_table))

# Calculate the average F1 score across all datasets
average_f1 = df_filtered.groupby(['alpha', 'epsilon'])['F1 Average'].mean().reset_index()
average_f1_pivot = average_f1.pivot(index='alpha', columns='epsilon', values='F1 Average')
contour_plots_data.append(('Average across Datasets', average_f1_pivot))

# Determine global min and max F1 scores for consistent color scaling across all plots
f1_min = min(data.min().min() for _, data in contour_plots_data)
f1_max = 1.0

# Create a new figure for the plots
fig, axes = plt.subplots(2, 3, figsize=(12, 7))

# Flatten the axes array for easy indexing
axes = axes.flatten()

# Plot each dataset with the unified color scaling
for i, (dataset, data) in enumerate(contour_plots_data):
    sns.heatmap(data, ax=axes[i], cmap="YlGnBu_r", cbar=False, linewidths=0, vmin=f1_min, vmax=f1_max)
    axes[i].invert_yaxis()
    axes[i].set_title(f'{dataset}')
    axes[i].set_ylabel('Alpha' if i % 3 == 0 else '')   # Only show the y axis label once for each row
    axes[i].set_xlabel('Epsilon' if i >= 3 else '')     # Only show the x axis label once for each column
    axes[i].set_yticks(axes[i].get_yticks(), axes[i].get_yticklabels(), rotation=0)

# Place a color bar at the right of the plots
cbar_ax = fig.add_axes([0.88, 0.15, 0.03, 0.7])
cbar = fig.colorbar(axes[0].collections[0], cax=cbar_ax)
cbar.set_label('F1 Score')
cbar.set_ticks(np.linspace(f1_min, f1_max, num=5))
cbar.ax.set_yticklabels([f'{tick:.0%}' for tick in cbar.get_ticks()])

# Add an overall title
fig.suptitle("HDBSCAN: Hyperparameter Tuning (Numerical Features)")

# Adjust layout for better fit and to make room for the color bar
fig.subplots_adjust(right=0.85)

# Download the plot as PDF
filename = "HDBSCAN Numerical Hyperparameter Optimization B.pdf"
plt.savefig(filename, bbox_inches='tight')
files.download(filename)

# Display the plots
plt.show()

In [ ]:
clusters_hdbscan = felix._cluster_features(felix._feature_embeddings)
features_hdbscan = felix._select_representative_features(felix._features, felix._feature_embeddings, clusters_hdbscan)
print(f"HDBSCAN selected {len(features_hdbscan.features)} features")

lr_prob, lr_score, rf_score = evaluate_feature_set([f.name for f in features_hdbscan.features], df_scores, d.y_test, cv_splits=5, random_state=42, lr_probs=True, lr_f1=True, rf_f1=True)
score_hdbscan = np.mean([lr_score, rf_score])
print(f"Score = {score_hdbscan}. LR_prob = {lr_prob}. LR_F1 = {lr_score}. RF_F1 = {rf_score}")

### K-means

In [ ]:
from sklearn.cluster import KMeans

scores_kmeans = []

for n_clusters in range(1, len(df_scores.columns)+1):
    kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init="auto")
    kmeans.fit(felix._feature_embeddings)
    clusters_kmeans = kmeans.labels_
    features_kmeans = felix._select_representative_features(felix._features, felix._feature_embeddings, clusters_kmeans)

    lr_prob, lr_score, rf_score = evaluate_feature_set([f.name for f in features_kmeans.features], df_scores, d.y_test, cv_splits=5, random_state=42, lr_probs=True, lr_f1=True, rf_f1=True)
    score = np.mean([lr_score, rf_score])
    scores_kmeans.append(score)

    log_results(d.id, "K-means", felix, [f.name for f in features_kmeans.features], rf_score, lr_score, lr_prob)

    print(f"n_features = {n_clusters}. Score = {score}. LR_prob = {lr_prob}. LR_F1 = {lr_score}. RF_F1 = {rf_score}")

### Hierarchical Agglomerative Clustering

In [ ]:
import numpy as np
from scipy.cluster.hierarchy import dendrogram, fcluster
from sklearn.cluster import AgglomerativeClustering

# Fit agglomerative hierarchical clustering
agg_clustering = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
agg_clustering.fit(felix._feature_embeddings)

# Create a linkage matrix from the AgglomerativeClustering output
num_samples = len(agg_clustering.labels_)
linkage_matrix = np.column_stack([
    agg_clustering.children_,
    agg_clustering.distances_,
    [num_samples + i for i in range(num_samples - 1)]
])

scores_agg = []

for n_clusters in range(1, len(df_scores.columns)+1):
    # Use fcluster to get the cluster labels based on a threshold for number of clusters
    threshold = linkage_matrix[:, 2][-(n_clusters - 2)]
    clusters_agg = fcluster(linkage_matrix, t=threshold, criterion="distance")

    features_agg = felix._select_representative_features(felix._features, felix._feature_embeddings, clusters_agg)

    lr_prob, lr_score, rf_score = evaluate_feature_set([f.name for f in features_agg.features], df_scores, d.y_test, cv_splits=5, random_state=42, lr_probs=True, lr_f1=True, rf_f1=True)
    score = np.mean([lr_score, rf_score])
    scores_agg.append(score)

    log_results(d.id, "Agglomerative Clustering", felix, [f.name for f in features_agg.features], rf_score, lr_score, lr_prob)

    print(f"n_features = {n_clusters}. Score = {score}. LR_prob = {lr_prob}. LR_F1 = {lr_score}. RF_F1 = {rf_score}")

### Random Feature Selection

Run random feature selection 5 times for each dataset and average the results

In [ ]:
import random

for i in range(5):
    scores_random = []

    # Create a random pertubation of the feature set
    features_shuffled = felix._features.copy(deep=True)
    random.shuffle(features_shuffled.features)

    for n_features in range(1, len(df_scores.columns)+1):
        feature_set = features_shuffled.copy(deep=True)
        feature_set.features = features_shuffled.features[:n_features]
        feature_names = [f.name for f in features_shuffled.features[:n_features]]

        lr_prob, lr_score, rf_score = evaluate_feature_set(feature_names, df_scores, d.y_test, cv_splits=5, random_state=42, lr_probs=True, lr_f1=True, rf_f1=True)
        score = np.mean([lr_score, rf_score])
        scores_random.append(score)

        log_results(d.id, "Random Feature Selection", felix, feature_names, rf_score, lr_score, lr_prob)

        print(f"iteration = {i + 1}. n_features = {n_features}. Score = {score}. LR_prob = {lr_prob}. LR_F1 = {lr_score}. RF_F1 = {rf_score}")

### Save Results

In [ ]:
from google.colab import files
import datetime
import json


json_string = json.dumps(results_log, indent=4)

# Store the results as a JSON
filename = f"{datetime.date.today().strftime('%Y_%m_%d')} - FELIX Consolidation Results {d.short_name}.json"
with open(filename, "w") as f:
    f.write(json_string)

# Download the results
files.download(filename)


# Convert the results log to a Pandas DataFrame
df_results = pd.DataFrame(results_log)

# Save the results as a CSV
filename = f"{datetime.date.today().strftime('%Y_%m_%d')} - FELIX Consolidation Results {d.short_name}.csv"
df_results.to_csv(filename, index=False)

# Download the results
files.download(filename)

In [ ]:
from google.colab import files
import json

# Upload the results JSON
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Load the scores
results_log = json.loads(uploaded[filename])
len(results_log)

### Plot Results

In [ ]:
import pandas as pd
import numpy as np

# Convert the results log to a Pandas DataFrame
df_results = pd.DataFrame(results_log)

# Calcualte average F1 scores from Random Forest and Logistic Regression
df_results["F1 Score"] = df_results.apply(lambda row: np.mean(row[["f1_rf", "f1_lr"]]), axis=1)

df_results

In [ ]:
df_results.groupby(["dataset", "method"]).count()

In [ ]:
import pandas as pd
import numpy as np

# Convert the results log to a Pandas DataFrame
df_results = pd.DataFrame(results_log)

# Calcualte average F1 scores from Random Forest and Logistic Regression
df_results["F1 Score"] = df_results.apply(lambda row: np.mean(row[["f1_rf", "f1_lr"]]), axis=1)

df_results

In [ ]:
df_results.groupby(["dataset", "method"]).count()

In [ ]:
# Simplify the dataset IDs for visualization
df_results["dataset"] = df_results["dataset"].map({
    "cardiffnlp/tweet_sentiment_multilingual": "Sentiment",
    "hate_speech18": "Hate Speech",
    "amazon_polarity": "Amazon",
    "GonzaloA/fake_news": "Fake News",
    "tum-nlp/IDMGSP": "Papers"
})

In [ ]:
# Set the order for the datasets
datasets_order = [
    "Sentiment",
    "Hate Speech",
    "Amazon",
    "Fake News",
    "Papers"
]

# Set the order for the legend
methods_order = [
    "Random Feature Selection",
    "Supervised Forward Selection",
    "Agglomerative Clustering",
    "K-means",
    "HDBSCAN"
]

# Define a color for each method
colors = {
    "Supervised Forward Selection": "#969696",
    "HDBSCAN": "#e6550d",
    "Agglomerative Clustering": "#6baed6",
    "K-means": "#74c476",
    "Random Feature Selection": "#d9d9d9"
}

In [ ]:
import matplotlib.ticker as mtick

# Group by 'n_features' and 'method' to calculate the mean of 'F1 Score' for duplicated entries (e.g., for random feature selection which has been executed multiple times)
df_adjusted = df_results.groupby(['n_features', 'method', 'dataset'])['F1 Score'].mean().reset_index()

# Drop entries for HDBSCAN with noise
df_adjusted = df_adjusted[df_adjusted["method"] != "HDBSCAN (w/ noise)"]
df_adjusted["method"] = df_adjusted["method"].replace("HDBSCAN (w/o noise)", "HDBSCAN")
df_adjusted = df_adjusted.reset_index()

# Create a grid of 2x3 for the scatter plots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 10))
axes = axes.flatten()

# Plot a scatter plot for each dataset
for i, dataset in enumerate(datasets_order):
    df_subset = df_adjusted[df_adjusted['dataset'] == dataset]
    for method in methods_order:
        df_method = df_subset[df_subset['method'] == method]
        axes[i].scatter(df_method['n_features'], df_method['F1 Score'], label=method, s=(15 if method == "HDBSCAN" else 3), color=colors[method], zorder=3)
    axes[i].set_title(dataset)
    axes[i].set_ylabel('F1 Score' if i % 2 == 0 else '')
    axes[i].set_xlabel('Number of Selected Features' if i > 3 else '')
    axes[i].set_xlim([0, max(df_subset['n_features'])])
    axes[i].yaxis.set_major_formatter(mtick.PercentFormatter(1.0, decimals=0))
    axes[i].grid(True, color='lightgrey', zorder=0)

# Plot a scotter plot of the average across all datasets
df_global_average = df_adjusted[df_adjusted["method"] != "HDBSCAN"].groupby(['n_features', 'method']).mean(numeric_only=True).reset_index()
hdbscan_average = df_adjusted[df_adjusted["method"] == "HDBSCAN"][["n_features", "F1 Score"]].mean()
least_features = df_adjusted[["dataset", "n_features"]].groupby("dataset").max()["n_features"].min()
for method in methods_order:
    df_method = df_global_average[df_global_average['method'] == method]
    axes[5].scatter(df_method['n_features'], df_method['F1 Score'], label=method, s=3, color=colors[method], zorder=3)
axes[5].scatter(hdbscan_average["n_features"], hdbscan_average["F1 Score"], label="HDBSCAN", s=15, color=colors["HDBSCAN"], zorder=3)
axes[5].set_title("Average across Datasets")
axes[5].set_xlim([0, least_features])
axes[5].set_xlabel('Number of Selected Features')
axes[5].yaxis.set_major_formatter(mtick.PercentFormatter(1.0, decimals=0))
axes[5].grid(True, color='lightgrey', zorder=0)

# Create a single legend for all plots
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncol=len(methods_order), bbox_to_anchor=(0.5, 1.02))

# Add a chart title
fig.suptitle('Feature Consolidation Performance (Categorical Features)', y=1.05)

# Adjust the layout
plt.tight_layout()

# Download the plot as PDF
filename = "Feature Consolidation Methods Categorical (All Datasets).pdf"
plt.savefig(filename, bbox_inches='tight')
files.download(filename)

plt.show()

In [ ]:
import matplotlib.pyplot as plt


# Group by 'n_features' and 'method' to calculate the mean of 'F1 Score' for duplicated entries (e.g., for random feature selection which has been executed multiple times)
df_adjusted = df_results.groupby(['n_features', 'method', 'dataset'])['F1 Score'].mean().reset_index()

# Drop entries for HDBSCAN with noise
df_adjusted = df_adjusted[df_adjusted["method"] != "HDBSCAN (w/ noise)"]
df_adjusted["method"] = df_adjusted["method"].replace("HDBSCAN (w/o noise)", "HDBSCAN")
df_adjusted = df_adjusted.reset_index()

# Calculate average results across all datasets
df_global_average = df_adjusted[df_adjusted["method"] != "HDBSCAN"].groupby(['n_features', 'method']).mean(numeric_only=True).reset_index()
least_features = df_adjusted[["dataset", "n_features"]].groupby("dataset").max()["n_features"].min()

# Create a plot
fig = plt.figure(figsize=(6, 3.5))

# Add results for each consolidation method
for method in methods_order:
    if method == "HDBSCAN":
        hdbscan_average = df_adjusted[df_adjusted["method"] == "HDBSCAN"][["n_features", "F1 Score"]].mean()
        plt.scatter(hdbscan_average["n_features"], hdbscan_average["F1 Score"], label="HDBSCAN", s=25, color=colors["HDBSCAN"], zorder=3)
    else:
        df_method = df_global_average[df_global_average['method'] == method]
        plt.scatter(df_method['n_features'], df_method['F1 Score'], label=method, s=3, color=colors[method], zorder=3)

# Format the plot
plt.title("Feature Consolidation Performance (Categorical Features)")
plt.xlim([0, least_features + 0.5])
plt.ylim([0.57, 0.87])
plt.xlabel('Number of Selected Features')
plt.ylabel('F1 Score')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1.0, decimals=0))
plt.grid(True, color='lightgrey', zorder=0)
plt.legend()
plt.tight_layout()

# Download the plot as PDF
filename = "Feature Consolidation Methods Categorical (Average).pdf"
plt.savefig(filename, bbox_inches='tight')
files.download(filename)

# Display the plot
plt.show()

In [ ]:
from google.colab import files

df_feature_selection = pd.DataFrame({"Forward Selection": scores_forward, "K-means": scores_kmeans, "Agglomerative": scores_agg, "Random": scores_random})

filename = "2023_11_02 - FELIX Feature Selection F1 Scores Fake News.csv"
df_feature_selection.to_csv(filename, index=False)

# Download the results
files.download(filename)

In [ ]:
from google.colab import files

# Upload the results
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Load the scores
df_feature_selection = pd.read_csv(filename)

scores_forward = df_feature_selection["Forward Selection"].tolist()
scores_kmeans = df_feature_selection["K-means"].tolist()
scores_agg = df_feature_selection["Agglomerative"].tolist()
scores_random = df_feature_selection["Random"].tolist()

## Experiment 5.2: Scoring Validity

In [ ]:
from collections import Counter

def gini_index(categories):
    # Count the occurrences of each category
    category_counts = Counter(categories)
    # Calculate the proportion of each category and square the proportions
    squared_proportions = [(count / len(categories)) ** 2 for count in category_counts.values()]
    # Sum the squared proportions and subtract from 1
    gini = 1 - sum(squared_proportions)
    return gini

# Given array
categories = ["A", "A", "C", "A", "B", "C"]
categories = ["A", "B", "C", "D", "E", "F", "G", "H"]

# Calculate Gini index
gini = gini_index(categories)

print(f"Gini index of the array: {gini}")


### Setup: Learn Numeric Feature Set

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import random

In [ ]:
# Select a dataset
d = dataset_list["papers"]
d

In [ ]:
# Fit FELIX to the dataset to learn features
felix = FELIX(
    context=d.context,
    temperature_scoring=0.0,
    discrete_features=False,
    verbose=True
)

felix.fit(d.X_train, d.y_train)

# Store the features learned by FELIX
feature_set = felix._features

### Case: Stable Feature Set

In [ ]:
# Get a random example
sample_row = d.X_test.sample(1, random_state=seed)

# Replicate the example 10 times and create a new dataframe
df = pd.concat([sample_row]*10, ignore_index=True)

# Score the example using FELIX
print("Scoring the same example 10 times ...")
df_scores_stable = felix.transform(df)

# Score different examples using FELIX
print("Scoring 10 different examples ...")
df_sample = d.X_test.sample(10, random_state=seed)
df_scores_general = felix.transform(df_sample)

In [ ]:
df_variance = pd.DataFrame({"Inner-sample": df_scores_stable.var(), "Inter-sample": df_scores_general.var()})

plt.figure(figsize=(8, 2))
ax = df_variance.boxplot(vert=False)
ax.set_xlabel("Score variance")
ax.set_title("Distribution of score variance inner-sample vs. inter-sample")
plt.tight_layout()
plt.show()

df_variance.describe()

Alternative plot showing the score distribution per feature:

In [ ]:
# Create a boxplot for the value distribution of each feature
plt.figure(figsize=(8, df_scores_stable.shape[1] / 5))
ax = df_scores_stable.boxplot(vert=False)
ax.set_xlabel('Value')
ax.set_title('Score distribution (stable order)')
plt.tight_layout()

plt.show()

### Case: Random Reordering

In [ ]:
# Get a random example
sample_row = d.X_test.sample(1, random_state=seed)

# Keep the scoring LLM stable with a 16K context window
felix.llm_scoring = "gpt-3.5-turbo-16k"
felix.verbose = True

# Score the original feature set 10 times
print("Scoring the original feature set ...")
felix._features = feature_set
results_rows_original = []
for _ in tqdm(range(10)):
    results_rows_original.append(felix.transform(sample_row))
df_scores_original = pd.concat(results_rows_original).reset_index(drop=True)

# Score a randomly reordered feature set 10 times
print("Scoring the randomly reordered feature set ...")
random.shuffle(felix._features.features) # Randomly reorders the feature set in place
results_rows_random = []
for _ in tqdm(range(10)):
    results_rows_random.append(felix.transform(sample_row))
df_scores_random = pd.concat(results_rows_random).reset_index(drop=True)

In [ ]:
# Perform t-test to see if changing the scoring LLM results in significantly different scores
def test_significance(df_model_a, df_model_b):
    p_values = {}
    for column in df_model_a.columns:
        _, p_value = stats.ttest_rel(df_model_a[column], df_model_b[column])
        p_values[column] = p_value
    return pd.Series(p_values)

p_values = test_significance(df_scores_original, df_scores_random)

# Excluding NaN values for plotting
p_values_cleaned = p_values.dropna()

# Plotting boxplots
plt.figure(figsize=(7, 1.5))
plt.boxplot([p_values_cleaned], vert=False, labels=["Random reordering"])
plt.xlabel("p-value")
plt.title("t-test for score differences when randomly reordering the feature set")
plt.tight_layout()
plt.show()

# Computing statistics
stats_random_reordering = {
    "Avg. p-value": [p_values_cleaned.mean()],
    "% of p-values < 0.01": [(p_values_cleaned < 0.01).mean() * 100],
    "% of p-values < 0.05": [(p_values_cleaned < 0.05).mean() * 100],
    "% of p-values < 0.1": [(p_values_cleaned < 0.1).mean() * 100]
}

# Creating a DataFrame
df_stats_random_reordering = pd.DataFrame(stats_random_reordering, index=["Random reordering"])
df_stats_random_reordering

Alternative experiment showing the large variance in scores when the feature set is randomly reschuffled in each scoring request:

In [ ]:
# Get a random example
sample_row = d.X_test.sample(1, random_state=seed)

# List to store the transformed rows
result_rows = []

for _ in range(10):
    # Randomly reorder the features
    random.shuffle(felix._features.features)
    print(f"Iteration {_}: first feature is {felix._features.features[0].name}")

    # Transform the sample row and append the result to the list
    transformed_row = felix.transform(sample_row)
    result_rows.append(transformed_row)

# Concatenate all resulting rows into a single dataframe
df_scores_random = pd.concat(result_rows).reset_index(drop=True)

In [ ]:
# Create a boxplot for the value distribution of each feature
plt.figure(figsize=(8, df_scores_random.shape[1] / 5))
ax = df_scores_random.boxplot(vert=False)
ax.set_xlabel('Value')
ax.set_title('Score distribution (random order)')
plt.tight_layout()

plt.show()

### Case: Splitting scoring requests

In [ ]:
# Get a random example
sample_row = d.X_test.sample(1, random_state=seed)

# Set scoring LLM to the 16K context variant of GPT-3.5 so that all features fit into the context of one request
felix.llm_scoring="gpt-3.5-turbo-16k"
felix.verbose=False

# Score the example 10 times with all features in one request
print("Scoring with all features in one request ...")
felix._features = feature_set
result_rows_no_split = []
for _ in tqdm(range(10)):
    # Transform the sample row and append the result to the list
    result_rows_no_split.append(felix.transform(sample_row))
df_scores_no_split = pd.concat(result_rows_no_split).reset_index(drop=True)

# Score the example 10 times with only the first half of the feature set
print("Scoring wih first half of the feature set ...")
felix._features = NumericalFeatureSet(features=feature_set.features[:len(feature_set.features)//2])
result_rows_split_a = []
for _ in tqdm(range(10)):
    # Transform the sample row and append the result to the list
    result_rows_split_a.append(felix.transform(sample_row))
df_scores_split_a = pd.concat(result_rows_split_a).reset_index(drop=True)

# Score the example 10 times with only the second half of the feature set
print("Scoring wih second half of the feature set ...")
felix._features = NumericalFeatureSet(features=feature_set.features[len(feature_set.features)//2:])
result_rows_split_b = []
for _ in tqdm(range(10)):
    # Transform the sample row and append the result to the list
    result_rows_split_b.append(felix.transform(sample_row))
df_scores_split_b = pd.concat(result_rows_split_b).reset_index(drop=True)

In [ ]:
# Perform t-test to see if splitting up scoring into multiple requests results in significantly different scores
def test_significance(df_split, df_no_split):
    p_values = {}
    for column in df_split.columns:
        if column in df_no_split.columns:
            _, p_value = stats.ttest_rel(df_split[column], df_no_split[column])
            p_values[column] = p_value
    return pd.Series(p_values)

p_values_split_a = test_significance(df_scores_split_a, df_scores_no_split)
p_values_split_b = test_significance(df_scores_split_b, df_scores_no_split)

# Excluding NaN values for plotting
p_values_a_cleaned = p_values_split_a.dropna()
p_values_b_cleaned = p_values_split_b.dropna()

# Plotting boxplots
plt.figure(figsize=(7, 3))
plt.boxplot([p_values_a_cleaned, p_values_b_cleaned], vert=False, labels=["First half", "Second half"])
plt.xlabel("p-value")
plt.title("t-test for score differences in first and second half of feature set vs. full feature set")
plt.tight_layout()
plt.show()

# Computing statistics
stats_split = {
    "Avg. p-value": [p_values_a_cleaned.mean(), p_values_b_cleaned.mean()],
    "% of p-values < 0.01": [(p_values_a_cleaned < 0.01).mean() * 100, (p_values_b_cleaned < 0.01).mean() * 100],
    "% of p-values < 0.05": [(p_values_a_cleaned < 0.05).mean() * 100, (p_values_b_cleaned < 0.05).mean() * 100],
    "% of p-values < 0.1": [(p_values_a_cleaned < 0.1).mean() * 100, (p_values_b_cleaned < 0.1).mean() * 100]
}

# Creating a DataFrame
df_stats_split = pd.DataFrame(stats_split, index=["First half", "Second half"])
df_stats_split

### Case: Changing LLM

In [ ]:
# Get a random example
sample_row = d.X_test.sample(1, random_state=seed)

# Limit the number of features to score due to small context window
max_features = 40
felix._features = NumericalFeatureSet(features=feature_set.features[:min(max_features, len(feature_set.features))])
felix.verbose=True

# Score the example 10 times with 4K context window
print("Scoring with with 4K context window ...")
felix.llm_scoring="gpt-3.5-turbo"
result_rows_4k = []
for _ in tqdm(range(10)):
    # Transform the sample row and append the result to the list
    result_rows_4k.append(felix.transform(sample_row))
df_scores_4k = pd.concat(result_rows_4k).reset_index(drop=True)

# Score the example 10 times with 16K context window
print("Scoring with with 16K context window ...")
felix.llm_scoring="gpt-3.5-turbo-16k"
result_rows_16k = []
for _ in tqdm(range(10)):
    # Transform the sample row and append the result to the list
    result_rows_16k.append(felix.transform(sample_row))
df_scores_16k = pd.concat(result_rows_16k).reset_index(drop=True)

In [ ]:
# Perform t-test to see if changing the scoring LLM results in significantly different scores
def test_significance(df_model_a, df_model_b):
    p_values = {}
    for column in df_model_a.columns:
        _, p_value = stats.ttest_rel(df_model_a[column], df_model_b[column])
        p_values[column] = p_value
    return pd.Series(p_values)

p_values = test_significance(df_scores_4k, df_scores_16k)

# Excluding NaN values for plotting
p_values_cleaned = p_values.dropna()

# Plotting boxplots
plt.figure(figsize=(7, 1.5))
plt.boxplot([p_values_cleaned], vert=False, labels=["Model change"])
plt.xlabel("p-value")
plt.title("t-test for score differences when changing the scoring LLM")
plt.tight_layout()
plt.show()

# Computing statistics
stats_model_change = {
    "Avg. p-value": [p_values_cleaned.mean()],
    "% of p-values < 0.01": [(p_values_cleaned < 0.01).mean() * 100],
    "% of p-values < 0.05": [(p_values_cleaned < 0.05).mean() * 100],
    "% of p-values < 0.1": [(p_values_cleaned < 0.1).mean() * 100]
}

# Creating a DataFrame
df_stats_model_change = pd.DataFrame(stats_model_change, index=["Model change"])
df_stats_model_change

### E2E Performance when Reschuffling

In [ ]:
d = dataset_list["papers"]
d

In [ ]:
callback = CustomFELIXCallback()
felix = FELIX(context=d.context_train, discrete_features=False, callback=callback, verbose=True)

felix.fit(d.X_train, d.y_train)

In [ ]:
original_features = felix._features
len(original_features.features)

In [ ]:
felix.reschuffle_features = True

# Transform data using feature reschuffling
df_scores_train_reschuffle = felix.transform(d.X_train)
df_scores_test_reschuffle = felix.transform(d.X_test)

In [ ]:
data_transformer = DataTransformer(dataset=d)
df_scores_train_reschuffle_imputed, df_scores_test_reschuffle_imputed = data_transformer.impute_missing_values(df_scores_train_reschuffle, df_scores_test_reschuffle)

columns = [col for col in df_scores_train_reschuffle_imputed.columns if col in df_scores_test_reschuffle_imputed.columns]

df_scores_train_reschuffle_imputed = df_scores_train_reschuffle_imputed[columns]
df_scores_test_reschuffle_imputed = df_scores_test_reschuffle_imputed[columns]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

rf = RandomForestClassifier()
rf.fit(df_scores_train_reschuffle_imputed, d.y_train)
y_pred_rf = rf.predict(df_scores_test_reschuffle_imputed)

print("F1 Random Forest:", f1_score(d.y_test, y_pred_rf, average="macro"))

lr = LogisticRegression(max_iter=1000)
lr.fit(df_scores_train_reschuffle_imputed, d.y_train)
y_pred_lr = lr.predict(df_scores_test_reschuffle_imputed)

print("F1 Logistic Regression:", f1_score(d.y_test, y_pred_lr, average="macro"))

In [ ]:
felix.reschuffle_features = False
felix._features = original_features

df_scores_train_stable = felix.transform(d.X_train)
df_scores_test_stable = felix.transform(d.X_test)

In [ ]:
data_transformer = DataTransformer(dataset=d)
df_scores_train_stable_imputed, df_scores_test_stable_imputed = data_transformer.impute_missing_values(df_scores_train_stable, df_scores_test_stable)

columns = [col for col in df_scores_train_stable_imputed.columns if col in df_scores_test_stable_imputed.columns]

df_scores_train_stable_imputed = df_scores_train_stable_imputed[columns]
df_scores_test_stable_imputed = df_scores_test_stable_imputed[columns]

In [ ]:
rf = RandomForestClassifier()
rf.fit(df_scores_train_stable_imputed, d.y_train)
y_pred_rf = rf.predict(df_scores_test_stable_imputed)

print("F1 Random Forest:", f1_score(d.y_test, y_pred_rf, average="macro"))

lr = LogisticRegression(max_iter=1000)
lr.fit(df_scores_train_stable_imputed, d.y_train)
y_pred_lr = lr.predict(df_scores_test_stable_imputed)

print("F1 Logistic Regression:", f1_score(d.y_test, y_pred_lr, average="macro"))